In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm # For a progress bar
import krippendorff
from sklearn.metrics import precision_score, recall_score, f1_score
import json
import ollama

#### Filter the data needed for news factor annotation

In [2]:
df = pd.read_csv("all_topic.csv")
df.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)
df.head()

,created_at,text,id,author_id,lang,username,text_clean,source_media,topic,probs
0,2021-12-31 12:06:05,'That was not the basis of the CDC's decision'...,1476887381336801280,2836421,en,MSNBC,that was not the basis of the cdcs decision dr...,1,13,0.707392
1,2021-12-30 02:46:02,'That was not the basis of the CDC's decision'...,1476384051154112512,2836421,en,MSNBC,that was not the basis of the cdcs decision dr...,1,13,0.698642
2,2021-12-29 13:31:07,'That was not the basis of the CDC's decision'...,1476184003526938624,2836421,en,MSNBC,that was not the basis of the cdcs decision dr...,1,13,0.708475
3,2021-12-29 06:31:02,'That was not the basis of the CDC's decision'...,1476078288661581824,2836421,en,MSNBC,that was not the basis of the cdcs decision dr...,1,13,0.713273
4,2021-08-25 19:34:01,Delta Air Lines CEO tells employees that they ...,1430614459077349376,2836421,en,MSNBC,delta air lines ceo tells employees that they ...,1,13,0.600323


In [3]:
df['created_at'] = pd.to_datetime(df['created_at'])
df['month_year'] = df['created_at'].dt.to_period('M')

In [4]:
# Sort by 'topic', 'month_year', and 'probs' in descending order
df_sorted = df.sort_values(by=['topic', 'month_year', 'probs'], ascending=[True, True, False])

# Group by 'topic' and 'month_year', and then select the top 5 rows
results = df_sorted.groupby(['topic', 'month_year']).head(5)

In [9]:
results = results[results['topic'] != -1]
results.reset_index(drop=True, inplace=True)

In [10]:
results.head(10)

,created_at,text,id,author_id,lang,username,text_clean,source_media,topic,probs,month_year
0,2021-01-31 21:20:06,A consortium of some of Canada’s largest compa...,1355989256666292224,807095,en,nytimes,a consoium of some of canada’s largest compani...,1,0,1.0,2021-01
1,2021-01-29 09:09:06,"CORONAVIRUS LATEST: An additional 3,872 fatali...",1355080518845530112,28785486,en,ABC,coronavirus latest an additional 3872 fataliti...,1,0,1.0,2021-01
2,2021-01-23 18:36:23,CORONAVIRUS LATEST: Week-to-week comparisons s...,1353048953642754048,28785486,en,ABC,coronavirus latest weektoweek comparisons show...,1,0,1.0,2021-01
3,2021-01-22 11:04:04,"CORONAVIRUS LATEST: An additional 3,955 fatali...",1352572737844371456,28785486,en,ABC,coronavirus latest an additional 3955 fataliti...,1,0,1.0,2021-01
4,2021-01-18 11:45:00,Groups nationwide are strategizing grassroots ...,1351133485742895104,15754281,en,USATODAY,groups nationwide are strategizing grassroots ...,1,0,1.0,2021-01
5,2021-02-04 00:00:00,"At this time last year, the U.S. was preparing...",1357116659115425792,28785486,en,ABC,at this time last year the us was preparing fo...,1,0,1.0,2021-02
6,2021-02-16 12:29:32,"RT @TODAYshow: “This drop is real, it’s not ju...",1361653941205938176,14173315,en,NBCNews,todayshow “this drop is real it’s not just an ...,1,0,1.0,2021-02
7,2021-02-26 07:46:14,🦠 Virus cases are down 30% nationwide compared...,1365206524684075008,5392522,en,NPR,virus cases are down 30 nationwide compared t...,1,0,1.0,2021-02
8,2021-02-23 03:57:49,🦠 Virus cases are down 40% nationwide compared...,1364061879459528704,5392522,en,NPR,virus cases are down 40 nationwide compared t...,1,0,1.0,2021-02
9,2021-02-20 03:40:08,🦠 Virus cases are down 44% nationwide compared...,1362970265379078144,5392522,en,NPR,virus cases are down 44 nationwide compared t...,1,0,1.0,2021-02


In [11]:
len(results)

9610

In [12]:
results.to_csv('news_factor.csv')

#### Get few-shot training samples 

In [9]:
df = pd.read_csv("newsfactor_coded.csv")
df.head()

,topic,topic_label,rep_doc,surprise,controversy,scope,positive_consequences,negative_consequences,elite_persons
0,0,COVID issues,read about workplace safety from the spread of...,0,0,0,0,0,0
1,0,COVID issues,disha ravi a 21yearold climate activist has be...,0,1,0,0,1,1
2,0,COVID issues,our thoughts are with everyone in india to he...,0,0,1,1,0,0
3,1,Netflix films,netflixfilm arianagrande x kidcudi just look u...,0,0,0,0,0,0
4,1,Netflix films,icymi judasandtheblackmessiah hits theaters an...,0,0,0,0,0,0


In [10]:
all_indices = set(df.index)
df1_indices = set()
df2_indices = set()

In [11]:
news_factors = ['surprise', 'controversy', 'scope', 'positive_consequences', 'negative_consequences', 'elite_persons']

In [12]:
for factor_col in news_factors:
    # Sample 3 '1's for the current factor_col
    available_ones = all_indices - df1_indices # Only consider indices not yet picked for df1
    current_col_ones_indices = df.index[(df.index.isin(available_ones)) & (df[factor_col] == 1)].tolist()
    
    num_to_sample_ones = min(3, len(current_col_ones_indices))
    if num_to_sample_ones > 0:
        sampled_ones = pd.Series(current_col_ones_indices).sample(n=num_to_sample_ones, random_state=1, replace=False).tolist()
        df1_indices.update(sampled_ones)
    if num_to_sample_ones < 3:
        print(f"Warning: For df1, could only get {num_to_sample_ones} '1's for '{factor_col}'.")

    # Sample 3 '0's for the current factor_col
    available_zeros = all_indices - df1_indices # Update available indices (df1_indices might have grown)
    current_col_zeros_indices = df.index[(df.index.isin(available_zeros)) & (df[factor_col] == 0)].tolist()

    num_to_sample_zeros = min(3, len(current_col_zeros_indices))
    if num_to_sample_zeros > 0:
        sampled_zeros = pd.Series(current_col_zeros_indices).sample(n=num_to_sample_zeros, random_state=2, replace=False).tolist()
        df1_indices.update(sampled_zeros)
    if num_to_sample_zeros < 3:
        print(f"Warning: For df1, could only get {num_to_sample_zeros} '0's for '{factor_col}'.")

df1 = df.loc[list(df1_indices)].copy()
print(f"df1 created with {len(df1)} rows.")

df1 created with 36 rows.


In [14]:
df1.to_csv('train.csv')

In [16]:
remaining_indices_after_df1 = all_indices - df1_indices
print(f"Rows remaining after df1: {len(remaining_indices_after_df1)}")

Rows remaining after df1: 480


In [17]:
for factor_col in news_factors:
    available_ones = remaining_indices_after_df1 - df2_indices # Only consider indices not yet picked for df2
    current_col_ones_indices = df.index[(df.index.isin(available_ones)) & (df[factor_col] == 1)].tolist()

    num_to_sample_ones = min(2, len(current_col_ones_indices))
    if num_to_sample_ones > 0:
        sampled_ones = pd.Series(current_col_ones_indices).sample(n=num_to_sample_ones, random_state=3, replace=False).tolist()
        df2_indices.update(sampled_ones)
    if num_to_sample_ones < 2:
        print(f"Warning: For df2, could only get {num_to_sample_ones} '1's for '{factor_col}'.")

print(f"df2 initial (guaranteed 1s) has {len(df2_indices)} unique rows.")

df2 initial (guaranteed 1s) has 11 unique rows.


In [18]:
remaining_indices_after_df2_initial = remaining_indices_after_df1 - df2_indices

In [19]:
num_additional_for_df2 = int(len(remaining_indices_after_df2_initial) * 0.15)

if num_additional_for_df2 > 0:
    additional_df2_indices = pd.Series(list(remaining_indices_after_df2_initial)).sample(n=num_additional_for_df2, random_state=4, replace=False).tolist()
    df2_indices.update(additional_df2_indices)
    print(f"Added {len(additional_df2_indices)} more rows to df2 (15% of remaining).")
else:
    print("No additional rows to add to df2 (15% was 0 or negative).")

df2 = df.loc[list(df2_indices)].copy()
print(f"df2 created with {len(df2)} rows.")


Added 70 more rows to df2 (15% of remaining).
df2 created with 81 rows.


In [20]:
df2.to_csv('val.csv')

In [21]:
remaining_indices_after_df2 = all_indices - df1_indices - df2_indices
print(f"Rows remaining after df2: {len(remaining_indices_after_df2)}")

Rows remaining after df2: 399


In [22]:
df3 = df.loc[list(remaining_indices_after_df2)].copy()
print(f"df3 created with {len(df3)} rows.")

df3 created with 399 rows.


In [24]:
df3.to_csv('test.csv')

### Elite persons

#### Prompt instruction model

In [2]:
# List locally available models (should show llama3 after you downloaded it)
print("Locally available Ollama models:")
print(ollama.list())

Locally available Ollama models:
models=[Model(model='llama3.1:latest', modified_at=datetime.datetime(2025, 7, 22, 16, 29, 12, 748502, tzinfo=TzInfo(+02:00)), digest='46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e', size=4920753328, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_K_M')), Model(model='deepseek-r1:latest', modified_at=datetime.datetime(2025, 7, 22, 14, 42, 30, 846205, tzinfo=TzInfo(+02:00)), digest='6995872bfe4c521a67b32da386cd21d5c6e819b6e0d62f79f64ec83be99f5763', size=5225376047, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='8.2B', quantization_level='Q4_K_M'))]


In [9]:
# Send a chat message to the model
messages = [
    {'role': 'user', 'content': """You are a highly precise prompt engineer. Trying to automatize part of the manual coding.
    Your goal is to generate a prompt that instructs an LLM to classify tweets.
    
    Instructions: 
        Output the prompt you generate in markdown. Output the prompt in a codeblock.   
        I want you to write a prompt for me that directly instructs an LLM to act as a classifier. 
        The classifier gets a tweet as input and codes whether one or more elite persons are presented in the tweet. 
        The prompt that you write should work as it is and will be followed up directly by the text that should be classified.
        Note that all coding from the classifier should be done on the basis of explicit content. Do not infer, imply, or assume identity if not explicitly stated. Thus, if a person is mentioned without it's role or identity being explicitly stated, it should not be coded as present.
        The classifier should be cautious of “hallucinations” - do not introduce information that is not present in the text.
        Make it clear to the classifier that the general mention of a role, e.g., this will help the presidents and military commandars, without clearly point to the speicific person should not be coded as elite person is present.
        Exception: The names "Trump" and "Biden" (and other US presidents by their last name only) are always considered explicit mentions of an elite person, regardless of further context.
        
        To leverage the few shot learning feature, make sure the definition, the examples, and its corresponding answers and rationales are passed to the classification model clearly to improve predictions: 
            Definition of "Elite Person":Mentions of well-known individuals. E.g., CEOs or influential political figures such as the U.S. President.
            Examples: 
            a) former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
            Answer: 1 - elite person is present. The tweet mentioned donald kohn and clearly stated the person's role as former fed vice chairman
            b) once you have talent offer upskilling or right skilling to help employees get better at their job or for the job they want to do pittorenatalie chief of enduring security framework national security agency womenatwork2021
            Answer: 1 - elite person is present. pittorenatalie is mentioned as chief of enduring security framework national security agency
            c) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
            Answer: 1 - elite person is present. jair bolsonaro is mentioned as the president of brazil.
            d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
            Answer: 0 - elite person is not present. No elite person is mentioned explicitly.
            e) we’re in the customer economy a great cx is essential for growth when you have consistent values both internally and externally you can quickly adjust to satisfy your evermore sophisticated clients learn how
            Answer: 0 - elite person is not present. No elite person is mentioned explicitly.
            f) in focus heimana reynolds for heimana reynolds the skateboard and park become ultimate platforms of creative and athletic expression watch heimana’s film from our exclusive documentary series now on twitter rlxteamusa teamusa tokyoolympics tokyo2020 olympics
            Answer: 0 - elite person is not present. No elite person is mentioned explicitly.
    
        The classification result MUST ONLY be a JSON object with two keys:
        1.  `"classification"`: an integer (0 or 1) representing the classification result.
        2.  `"cot"`: a string containing the brief chain of thought/reasoning for the classification.

    Always provide your chain of thought for generating the prompts but keep it short.
"""},
]

response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response:")
print(response['message']['content'])

# You can also use the generate API for simple text generation
# response_generate = ollama.generate(model='llama3', prompt='Tell me a short story about a cat.')
# print("\nGenerated Story:")
# print(response_generate['response'])


Model Response:
**Prompt**
```markdown
Classify tweets as having an elite person mentioned or not. Elite persons are well-known individuals, such as CEOs or influential political figures like US Presidents.

Given a tweet, classify whether one or more elite persons are presented in the tweet.

Note that:

*   Only explicit mentions of roles or identities are considered.
*   General mentions of roles without clear pointing to specific individuals should not be coded as elite person is present.
*   Exception: Names "Trump" and "Biden" (and other US Presidents by their last name only) are always considered explicit mentions of an elite person, regardless of further context.

Definition:
Elite Person: Mentions of well-known individuals. E.g., CEOs or influential political figures such as the U.S. President.

Examples:

a) former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
Answer: 1 

#### Try the prompt

In [28]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more elite persons are presented in the tweet.

Note that:
*   Only explicit mentions of roles or identities are considered.
*   Exception: The name "Trump" and "Biden" (and other US Presidents BY THEIR LAST NAME ONLY) are always considered explicit mentions of an elite person, regardless of further context.
*   General mentions of roles, without clear pointing to specific individuals should be coded as elite person is NOT present.

Definition:
Elite Person: Mentions of well-known individuals. E.g., CEOs or influential political figures such as the U.S. President.

Examples:
a) former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
Answer: 1 - elite person is present.
Reasoning: The tweet mentioned donald kohn and clearly stated the person's role as former fed vice chairman.
b) once you have talent offer upskilling or right skilling to help employees get better at their job or for the job they want to do pittorenatalie chief of enduring security framework national security agency womenatwork2021
Answer: 1 - elite person is present.
Reasoning: pittorenatalie is mentioned as chief of enduring security framework national security agency
c) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - elite person is present.
Reasoning: jair bolsonaro is mentioned as the president of brazil.
d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
e) we’re in the customer economy a great cx is essential for growth when you have consistent values both internally and externally you can quickly adjust to satisfy your evermore sophisticated clients learn how
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
f) in focus heimana reynolds for heimana reynolds the skateboard and park become ultimate platforms of creative and athletic expression watch heimana’s film from our exclusive documentary series now on twitter rlxteamusa teamusa tokyoolympics tokyo2020 olympics
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
moisturize skin while you bathe skinsosoft
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 0,
  "cot": "No elite person is mentioned explicitly."
}


In [29]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more elite persons are presented in the tweet.

Note that:
*   Only explicit mentions of roles or identities are considered.
*   Exception: The name "Trump" and "Biden" (and other US Presidents BY THEIR LAST NAME ONLY) are always considered explicit mentions of an elite person, regardless of further context.
*   General mentions of roles, without clear pointing to specific individuals should be coded as elite person is NOT present.

Definition:
Elite Person: Mentions of well-known individuals. E.g., CEOs or influential political figures such as the U.S. President.

Examples:
a) former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
Answer: 1 - elite person is present.
Reasoning: The tweet mentioned donald kohn and clearly stated the person's role as former fed vice chairman.
b) once you have talent offer upskilling or right skilling to help employees get better at their job or for the job they want to do pittorenatalie chief of enduring security framework national security agency womenatwork2021
Answer: 1 - elite person is present.
Reasoning: pittorenatalie is mentioned as chief of enduring security framework national security agency
c) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - elite person is present.
Reasoning: jair bolsonaro is mentioned as the president of brazil.
d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
e) we’re in the customer economy a great cx is essential for growth when you have consistent values both internally and externally you can quickly adjust to satisfy your evermore sophisticated clients learn how
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
f) in focus heimana reynolds for heimana reynolds the skateboard and park become ultimate platforms of creative and athletic expression watch heimana’s film from our exclusive documentary series now on twitter rlxteamusa teamusa tokyoolympics tokyo2020 olympics
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
by building underground transmissions we are building a system that promotes clean and reliable energy learn more about this project in the latest episode of our plugged in podcast
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 0,
  "cot": "No elite person is mentioned explicitly."
}


In [30]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more elite persons are presented in the tweet.

Note that:
*   Only explicit mentions of roles or identities are considered.
*   Exception: The name "Trump" and "Biden" (and other US Presidents BY THEIR LAST NAME ONLY) are always considered explicit mentions of an elite person, regardless of further context.
*   General mentions of roles, without clear pointing to specific individuals should be coded as elite person is NOT present.

Definition:
Elite Person: Mentions of well-known individuals. E.g., CEOs or influential political figures such as the U.S. President.

Examples:
a) former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
Answer: 1 - elite person is present.
Reasoning: The tweet mentioned donald kohn and clearly stated the person's role as former fed vice chairman.
b) once you have talent offer upskilling or right skilling to help employees get better at their job or for the job they want to do pittorenatalie chief of enduring security framework national security agency womenatwork2021
Answer: 1 - elite person is present.
Reasoning: pittorenatalie is mentioned as chief of enduring security framework national security agency
c) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - elite person is present.
Reasoning: jair bolsonaro is mentioned as the president of brazil.
d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
e) we’re in the customer economy a great cx is essential for growth when you have consistent values both internally and externally you can quickly adjust to satisfy your evermore sophisticated clients learn how
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
f) in focus heimana reynolds for heimana reynolds the skateboard and park become ultimate platforms of creative and athletic expression watch heimana’s film from our exclusive documentary series now on twitter rlxteamusa teamusa tokyoolympics tokyo2020 olympics
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
hong kongs prodemocracy newspaper apple daily is shutting down after 26 years the move comes after the government froze its bank accounts and arrested several top executives including the editorinchief
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
    "classification": 1,
    "cot": "the tweet mentioned top executives, which implies well-known individuals"
}


In [31]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more elite persons are presented in the tweet.

Note that:
*   Only explicit mentions of roles or identities are considered.
*   Exception: The name "Trump" and "Biden" (and other US Presidents BY THEIR LAST NAME ONLY) are always considered explicit mentions of an elite person, regardless of further context.
*   General mentions of roles, without clear pointing to specific individuals should be coded as elite person is NOT present.

Definition:
Elite Person: Mentions of well-known individuals. E.g., CEOs or influential political figures such as the U.S. President.

Examples:
a) former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
Answer: 1 - elite person is present.
Reasoning: The tweet mentioned donald kohn and clearly stated the person's role as former fed vice chairman.
b) once you have talent offer upskilling or right skilling to help employees get better at their job or for the job they want to do pittorenatalie chief of enduring security framework national security agency womenatwork2021
Answer: 1 - elite person is present.
Reasoning: pittorenatalie is mentioned as chief of enduring security framework national security agency
c) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - elite person is present.
Reasoning: jair bolsonaro is mentioned as the president of brazil.
d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
e) we’re in the customer economy a great cx is essential for growth when you have consistent values both internally and externally you can quickly adjust to satisfy your evermore sophisticated clients learn how
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
f) in focus heimana reynolds for heimana reynolds the skateboard and park become ultimate platforms of creative and athletic expression watch heimana’s film from our exclusive documentary series now on twitter rlxteamusa teamusa tokyoolympics tokyo2020 olympics
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
amazon unbound by brad stone is on its surface a business book writes benyt in his review for nytimesbooks its also very much a biography of jeff bezos that makes for a dense at times juicy tour of the company bezos built
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 1,
  "cot": "benyt and jeff bezos are explicitly mentioned, indicating an elite person is present."
}


#### Validate the performance

In [57]:
MESSAGE = """Given a tweet, classify whether one or more elite persons are presented in the tweet.

Note that:
*   Only explicit mentions of roles or identities are considered. The roles and identities should have far-reaching influence or power. 
*   Do not infer, imply, or assume identity if not explicitly stated.
*   Do not introduce information that is not present in the text.
*   Mention of company names, even if they are well-known company, should NOT be counted elite persons.
*   Exception: The name "Trump" and "Biden" (and other US Presidents BY THEIR LAST NAME ONLY), and Dr. Martin Luther King, Jr are always considered explicit mentions of an elite person, regardless of further context.

Definition:
Elite Person: Mentions of well-known individuals. E.g., Movie stars, CEOs, or influential political figures such as governmental officials.

Examples:
a) former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
Answer: 1 - elite person is present.
Reasoning: The tweet mentioned donald kohn and clearly stated the person's role as former fed vice chairman.
b) once you have talent offer upskilling or right skilling to help employees get better at their job or for the job they want to do pittorenatalie chief of enduring security framework national security agency womenatwork2021
Answer: 1 - elite person is present.
Reasoning: pittorenatalie is mentioned as chief of enduring security framework national security agency
c) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - elite person is present.
Reasoning: jair bolsonaro is mentioned as the president of brazil.
d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
e) we’re in the customer economy a great cx is essential for growth when you have consistent values both internally and externally you can quickly adjust to satisfy your evermore sophisticated clients learn how
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
f) in focus heimana reynolds for heimana reynolds the skateboard and park become ultimate platforms of creative and athletic expression watch heimana’s film from our exclusive documentary series now on twitter rlxteamusa teamusa tokyoolympics tokyo2020 olympics
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet: 
'{tweet_text}'
"""

In [63]:
val = pd.read_csv("val.csv")
val.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [11]:
def classify_tweet_with_ollama(tweet_text, model_name="llama3.1"):
    """
    Sends a tweet to the Ollama model for classification and extracts the result and CoT from JSON.
    """
    full_prompt = MESSAGE.format(tweet_text=tweet_text)
    try:
        response = ollama.chat(
            model=model_name,
            messages=[{'role': 'user', 'content': full_prompt}],
            options={'temperature': 0.0}
        )
        content = response['message']['content'].strip()

        classification = np.nan
        cot = ""

        try:
            parsed_json = json.loads(content)
            # Extract values from the parsed JSON
            if 'classification' in parsed_json:
                classification = int(parsed_json['classification'])
            if 'cot' in parsed_json:
                cot = parsed_json['cot']
            else:
                print(f"Warning: No JSON found in LLM output for tweet: {tweet_text[:50]}...")
                print(f"LLM output (first 100 chars): {content[:100]}...")
                cot = content # Store full content if JSON not found

        except (json.JSONDecodeError, ValueError) as e:
            print(f"Warning: JSON parsing/conversion error for tweet: {tweet_text[:50]}... Error: {e}")
            print(f"LLM raw output: {content[:100]}...")
            cot = content # Store raw content if parsing failed

        return classification, cot

    except Exception as e:
        print(f"Error classifying tweet: {tweet_text[:50]}... Error: {e}")
        return np.nan, f"Error during classification: {e}"

In [59]:
llm_results = []
llm_cots = []
model_to_use = "llama3.1" 

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet_text = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(tweet_text, model_name=model_to_use)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['elite_persons_llm1'] = llm_results
val['elite_persons_llm_cot1'] = llm_cots


Classifying Tweets:   0%|          | 0/81 [00:00<?, ?it/s]Classifying Tweets: 100%|██████████| 81/81 [02:58<00:00,  2.20s/it]


In [66]:
llm_results = []
llm_cots = []
model_to_use = "llama3.1" 

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet_text = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(tweet_text, model_name=model_to_use)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['elite_persons_llm2'] = llm_results
val['elite_persons_llm_cot2'] = llm_cots


Classifying Tweets: 100%|██████████| 81/81 [02:50<00:00,  2.10s/it]


Intercoder reliability between 2 runs of llm

In [67]:
val.groupby(['elite_persons_llm1','elite_persons_llm2'])['topic'].count()

elite_persons_llm1  elite_persons_llm2
0                   0                     66
1                   1                     15
Name: topic, dtype: int64

In [68]:
krippendorff.alpha((val['elite_persons_llm1'].values.tolist(), 
                    val['elite_persons_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

Compare performance between llm and human coder

In [64]:
val.groupby(['elite_persons','elite_persons_llm1'])['topic'].count()

elite_persons  elite_persons_llm1
0              0                     65
               1                      3
1              0                      1
               1                     12
Name: topic, dtype: int64

In [75]:
precision = precision_score(val['elite_persons'], val['elite_persons_llm1'])
recall = recall_score(val['elite_persons'], val['elite_persons_llm1'])
f1 = f1_score(val['elite_persons'], val['elite_persons_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 0.80
Recall: 0.92
F1-score: 0.86


In [69]:
val.to_csv('val.csv')

#### Test the final prompt

In [ ]:
MESSAGE = """Given a tweet, classify whether one or more elite persons are presented in the tweet.

Note that:
*   Only explicit mentions of roles or identities are considered. The roles and identities should have far-reaching influence or power. 
*   Do not infer, imply, or assume identity if not explicitly stated.
*   Do not introduce information that is not present in the text.
*   Mention of company names, even if they are well-known company, should NOT be counted elite persons.
*   Exception: The name "Trump" and "Biden" (and other US Presidents BY THEIR LAST NAME ONLY), and Dr. Martin Luther King, Jr are always considered explicit mentions of an elite person, regardless of further context.

Definition:
Elite Person: Mentions of well-known individuals. E.g., Movie stars, CEOs, or influential political figures such as governmental officials.

Examples:
a) former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
Answer: 1 - elite person is present.
Reasoning: The tweet mentioned donald kohn and clearly stated the person's role as former fed vice chairman.
b) once you have talent offer upskilling or right skilling to help employees get better at their job or for the job they want to do pittorenatalie chief of enduring security framework national security agency womenatwork2021
Answer: 1 - elite person is present.
Reasoning: pittorenatalie is mentioned as chief of enduring security framework national security agency
c) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - elite person is present.
Reasoning: jair bolsonaro is mentioned as the president of brazil.
d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
e) we’re in the customer economy a great cx is essential for growth when you have consistent values both internally and externally you can quickly adjust to satisfy your evermore sophisticated clients learn how
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
f) in focus heimana reynolds for heimana reynolds the skateboard and park become ultimate platforms of creative and athletic expression watch heimana’s film from our exclusive documentary series now on twitter rlxteamusa teamusa tokyoolympics tokyo2020 olympics
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet: 
'{tweet_text}'
"""

In [53]:
test = pd.read_csv("test.csv")
test.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [ ]:
llm_results = []
llm_cots = []
model_to_use = "llama3.1" 

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet_text = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(tweet_text, model_name=model_to_use)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['elite_persons_llm1'] = llm_results
test['elite_persons_llm_cot1'] = llm_cots

In [ ]:
llm_results = []
llm_cots = []
model_to_use = "llama3.1" 

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet_text = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(tweet_text, model_name=model_to_use)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['elite_persons_llm2'] = llm_results
test['elite_persons_llm_cot2'] = llm_cots

In [54]:
test.groupby(['elite_persons_llm1','elite_persons_llm2'])['topic'].count()

elite_persons_llm1  elite_persons_llm2
0                   0                     332
1                   1                      67
Name: topic, dtype: int64

In [55]:
krippendorff.alpha((test['elite_persons_llm1'].values.tolist(), 
                    test['elite_persons_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

In [56]:
test.groupby(['elite_persons','elite_persons_llm1'])['topic'].count()

elite_persons  elite_persons_llm1
0              0                     331
               1                      19
1              0                       1
               1                      48
Name: topic, dtype: int64

In [59]:
precision = precision_score(test['elite_persons'], test['elite_persons_llm1'])
recall = recall_score(test['elite_persons'], test['elite_persons_llm1'])
f1 = f1_score(test['elite_persons'], test['elite_persons_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 0.72
Recall: 0.98
F1-score: 0.83


In [ ]:
test.to_csv('test.csv')

#### Make predictions for the full dataset

In [ ]:
nf = pd.read_csv("newsfactor.csv")
nf.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [ ]:
MESSAGE = """Given a tweet, classify whether one or more elite persons are presented in the tweet.

Note that:
*   Only explicit mentions of roles or identities are considered. The roles and identities should have far-reaching influence or power. 
*   Do not infer, imply, or assume identity if not explicitly stated.
*   Do not introduce information that is not present in the text.
*   Mention of company names, even if they are well-known company, should NOT be counted elite persons.
*   Exception: The name "Trump" and "Biden" (and other US Presidents BY THEIR LAST NAME ONLY), and Dr. Martin Luther King, Jr are always considered explicit mentions of an elite person, regardless of further context.

Definition:
Elite Person: Mentions of well-known individuals. E.g., Movie stars, CEOs, or influential political figures such as governmental officials.

Examples:
a) former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
Answer: 1 - elite person is present.
Reasoning: The tweet mentioned donald kohn and clearly stated the person's role as former fed vice chairman.
b) once you have talent offer upskilling or right skilling to help employees get better at their job or for the job they want to do pittorenatalie chief of enduring security framework national security agency womenatwork2021
Answer: 1 - elite person is present.
Reasoning: pittorenatalie is mentioned as chief of enduring security framework national security agency
c) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - elite person is present.
Reasoning: jair bolsonaro is mentioned as the president of brazil.
d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
e) we’re in the customer economy a great cx is essential for growth when you have consistent values both internally and externally you can quickly adjust to satisfy your evermore sophisticated clients learn how
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.
f) in focus heimana reynolds for heimana reynolds the skateboard and park become ultimate platforms of creative and athletic expression watch heimana’s film from our exclusive documentary series now on twitter rlxteamusa teamusa tokyoolympics tokyo2020 olympics
Answer: 0 - elite person is not present.
Reasoning: No elite person is mentioned explicitly.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet: 
'{tweet_text}'
"""

In [ ]:
llm_results = []
llm_cots = []
model_to_use = "llama3.1" 

for index, row in tqdm(nf.iterrows(), total=len(nf), desc="Classifying Tweets"):
    tweet_text = row['text_clean']
    llm_classification, llm_cot = classify_tweet_with_ollama(tweet_text, model_name=model_to_use)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

nf['elite_persons_llm'] = llm_results
nf['elite_persons_llm_cot'] = llm_cots

In [ ]:
nf.to_csv('newsfactor.csv')

### Negative consequences

#### Prompt instruction model

In [3]:
# List locally available models (should show llama3 after you downloaded it)
print("Locally available Ollama models:")
print(ollama.list())

Locally available Ollama models:
models=[Model(model='llama3.1:latest', modified_at=datetime.datetime(2025, 7, 22, 16, 29, 12, 748502, tzinfo=TzInfo(+02:00)), digest='46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e', size=4920753328, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_K_M')), Model(model='deepseek-r1:latest', modified_at=datetime.datetime(2025, 7, 22, 14, 42, 30, 846205, tzinfo=TzInfo(+02:00)), digest='6995872bfe4c521a67b32da386cd21d5c6e819b6e0d62f79f64ec83be99f5763', size=5225376047, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='8.2B', quantization_level='Q4_K_M'))]


In [4]:
messages = [
    {'role': 'user', 'content': """You are a highly precise prompt engineer. Trying to automatize part of the manual coding.
    Your goal is to generate a prompt that instructs an LLM to classify tweets.
    Here's a prompt you wrote for the varialbe elite persons. Now you need to adapt this prompt for the variable negative consequences.
    
        Given a tweet, classify whether one or more elite persons are presented in the tweet.

        Note that:
        *   Only explicit mentions of roles or identities are considered. The roles and identities should have far-reaching influence or power. 
        *   Do not infer, imply, or assume identity if not explicitly stated.
        *   Do not introduce information that is not present in the text.
        *   Mention of company names, even if they are well-known company, should NOT be counted elite persons.
        *   Exception: The name "Trump" and "Biden" (and other US Presidents BY THEIR LAST NAME ONLY), and Dr. Martin Luther King, Jr are always considered explicit mentions of an elite person, regardless of further context.

        Definition:
        Elite Person: Mentions of well-known individuals. E.g., Movie stars, CEOs, or influential political figures such as governmental officials.

        Examples:
        a) former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
        Answer: 1 - elite person is present.
        Reasoning: The tweet mentioned donald kohn and clearly stated the person's role as former fed vice chairman.
        b) once you have talent offer upskilling or right skilling to help employees get better at their job or for the job they want to do pittorenatalie chief of enduring security framework national security agency womenatwork2021
        Answer: 1 - elite person is present.
        Reasoning: pittorenatalie is mentioned as chief of enduring security framework national security agency
        c) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
        Answer: 1 - elite person is present.
        Reasoning: jair bolsonaro is mentioned as the president of brazil.
        d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
        Answer: 0 - elite person is not present.
        Reasoning: No elite person is mentioned explicitly.
        e) we’re in the customer economy a great cx is essential for growth when you have consistent values both internally and externally you can quickly adjust to satisfy your evermore sophisticated clients learn how
        Answer: 0 - elite person is not present.
        Reasoning: No elite person is mentioned explicitly.
        f) in focus heimana reynolds for heimana reynolds the skateboard and park become ultimate platforms of creative and athletic expression watch heimana’s film from our exclusive documentary series now on twitter rlxteamusa teamusa tokyoolympics tokyo2020 olympics
        Answer: 0 - elite person is not present.
        Reasoning: No elite person is mentioned explicitly.

        THE ONLY OUTPUT should be a JSON object with two keys:
        1. `classification`: an integer (0 or 1) representing the classification result.
        2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

        Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

        Input tweet: 
        '{tweet_text}'
    
    Here's the definiation of the variable negative consequences: Negative outcomes for one or more actors explicitly mentioned. E.g., damage from extreme weather or legal penalties faced by a corporation.
    Here're the examples for few shot learning:
        a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
        Answer: 1 - negative consequence is present.
        Reasoning: The tweet explicitly menionted a rising deforestation in the amazon, which can be seen as negative outcomes for the environment.
        b) this will likely get little international media attention but it is a very very dark moment for press freedom in hk an apple daily columnist tweeted
        Answer: 1 - negative consequence is present.
        Reasoning: The tweet explicitly mentioned a very very dark moment for press freedom in hk, showing the negative consequence on the press in HK.
        c) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
        Answer: 1 - negative consequence is present.
        Reasoning: The tweet explicitly mentioned the sudden lift of the COVID restriction has negatively impacted the ability of private companies to protect their staffers and customers
        d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
        Answer: 0 - negative consequence is not present.
        Reasoning: No explicit mention of negative consequence.
        e) once you have talent offer upskilling or right skilling to help employees get better at their job or for the job they want to do pittorenatalie chief of enduring security framework national security agency womenatwork2021
        Answer: 0 - negative consequence is not present.
        Reasoning: No explicit mention of negative consequence.
        f) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
        Answer: 0 - negative consequence is not present.
        Reasoning: No explicit mention of negative consequence.
    
    Always provide your chain of thought for generating the prompts but keep it short.
"""},
]

response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response:")
print(response['message']['content'])


Model Response:
Here's the prompt I came up with to classify tweets based on the presence of negative consequences:

"Given a tweet, classify whether one or more negative consequences are presented in the tweet. 

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a detrimental effect or penalty.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.
*   Mention of company names, even if they are well-known companies, should NOT be counted as negative consequences.
*   Exception: The mention of damage from extreme weather events, legal penalties faced by a corporation, or financial losses should always be considered explicit mentions of a negative consequence.

Definition:
Negative Consequence: Mentions of adverse outcomes for one or more actors. E.g., damage from extreme weather or legal penalties faced by a corporation.

Examples:

a) presi

#### Try the prompt

In [5]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more negative consequences are presented in the tweet. 

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a detrimental effect or penalty.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Negative Consequence: Mentions of adverse outcomes for one or more actors. E.g., damage from extreme weather or legal penalties faced by a corporation.

Examples:

a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a rising deforestation in the amazon, which can be seen as negative outcomes for the environment.

b) this will likely get little international media attention but it is a very very dark moment for press freedom in hk an apple daily columnist tweeted
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a very very dark moment for press freedom in hk, showing the negative consequence on the press in HK.

c) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned the sudden lift of the COVID restriction has negatively impacted the ability of private companies to protect their staffers and customers.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - negative consequence is not present.
Reasoning: No explicit mention of negative consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

Input tweet:
     former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 1,
  "cot": "The tweet explicitly mentioned a rising threat of faster inflation, which can be seen as negative outcomes for the economy."
}


In [6]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more negative consequences are presented in the tweet. 

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a detrimental effect or penalty.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Negative Consequence: Mentions of adverse outcomes for one or more actors. E.g., damage from extreme weather or legal penalties faced by a corporation.

Examples:

a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a rising deforestation in the amazon, which can be seen as negative outcomes for the environment.

b) this will likely get little international media attention but it is a very very dark moment for press freedom in hk an apple daily columnist tweeted
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a very very dark moment for press freedom in hk, showing the negative consequence on the press in HK.

c) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned the sudden lift of the COVID restriction has negatively impacted the ability of private companies to protect their staffers and customers.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - negative consequence is not present.
Reasoning: No explicit mention of negative consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

Input tweet:
    the online trading app robinhood promised to wrest the stock market away from wall street’s gatekeepers — making it as easy to put millions at risk as it is to summon an uber it turns out robinhood was at the mercy of the industry it had vowed to upend
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 1,
  "cot": "The tweet explicitly mentioned that the online trading app Robinhood was at the mercy of the industry it had vowed to upend, showing a negative consequence for its original promise."
}


In [7]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more negative consequences are presented in the tweet. 

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a detrimental effect or penalty.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Negative Consequence: Mentions of adverse outcomes for one or more actors. E.g., damage from extreme weather or legal penalties faced by a corporation.

Examples:

a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a rising deforestation in the amazon, which can be seen as negative outcomes for the environment.

b) this will likely get little international media attention but it is a very very dark moment for press freedom in hk an apple daily columnist tweeted
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a very very dark moment for press freedom in hk, showing the negative consequence on the press in HK.

c) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned the sudden lift of the COVID restriction has negatively impacted the ability of private companies to protect their staffers and customers.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - negative consequence is not present.
Reasoning: No explicit mention of negative consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

Input tweet:
    the road to global hcv elimination is possible see how working together during covid will help us eliminate hcv worldwide
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 0,
  "cot": "No explicit mention of negative consequence."
}


In [8]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more negative consequences are presented in the tweet. 

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a detrimental effect or penalty.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Negative Consequence: Mentions of adverse outcomes for one or more actors. E.g., damage from extreme weather or legal penalties faced by a corporation.

Examples:

a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a rising deforestation in the amazon, which can be seen as negative outcomes for the environment.

b) this will likely get little international media attention but it is a very very dark moment for press freedom in hk an apple daily columnist tweeted
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a very very dark moment for press freedom in hk, showing the negative consequence on the press in HK.

c) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned the sudden lift of the COVID restriction has negatively impacted the ability of private companies to protect their staffers and customers.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - negative consequence is not present.
Reasoning: No explicit mention of negative consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

Input tweet:
    were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other food justice work at bhem
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 0,
  "cot": "No explicit mention of negative consequence."
}


#### Validate the performance

In [21]:
MESSAGE = """Given a tweet, classify whether one or more negative consequences are presented in the tweet. 

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a detrimental effect or penalty.
*   Illnesses in a SPECIFIC PERSON or a SPECIFIC GROUP OF PEOPLE, and future challenges facing a company or industry can also be considered negative consequences.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Negative Consequence: Mentions of adverse outcomes for one or more actors. E.g., damage from extreme weather or legal penalties faced by a corporation.

Examples:

a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a rising deforestation in the amazon, which can be seen as negative outcomes for the environment.

b) this will likely get little international media attention but it is a very very dark moment for press freedom in hk an apple daily columnist tweeted
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a very very dark moment for press freedom in hk, showing the negative consequence on the press in HK.

c) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned the sudden lift of the COVID restriction has negatively impacted the ability of private companies to protect their staffers and customers.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - negative consequence is not present.
Reasoning: No explicit mention of negative consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [26]:
val = pd.read_csv("val.csv")
val.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [23]:
llm_results = []
llm_cots = []
model_to_use = "llama3.1" 

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet_text = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(tweet_text, model_name=model_to_use)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['negative_consequences_llm1'] = llm_results
val['negative_consequences_llm_cot1'] = llm_cots

Classifying Tweets: 100%|██████████| 81/81 [02:37<00:00,  1.94s/it]


In [27]:
llm_results = []
llm_cots = []
model_to_use = "llama3.1" 

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet_text = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(tweet_text, model_name=model_to_use)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['negative_consequences_llm2'] = llm_results
val['negative_consequences_llm_cot2'] = llm_cots

Classifying Tweets: 100%|██████████| 81/81 [02:32<00:00,  1.89s/it]


In [28]:
val.groupby(['negative_consequences_llm1','negative_consequences_llm2'])['topic'].count()

negative_consequences_llm1  negative_consequences_llm2
0                           0                             69
1                           1                             12
Name: topic, dtype: int64

In [29]:
krippendorff.alpha((val['negative_consequences_llm1'].values.tolist(), 
                    val['negative_consequences_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

In [30]:
val.groupby(['negative_consequences','negative_consequences_llm1'])['topic'].count()

negative_consequences  negative_consequences_llm1
0                      0                             66
1                      0                              3
                       1                             12
Name: topic, dtype: int64

In [32]:
precision = precision_score(val['negative_consequences'], val['negative_consequences_llm1'])
recall = recall_score(val['negative_consequences'], val['negative_consequences_llm1'])
f1 = f1_score(val['negative_consequences'], val['negative_consequences_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 1.00
Recall: 0.80
F1-score: 0.89


In [33]:
val.to_csv('val.csv')

#### Test the final prompt

In [42]:
MESSAGE = """Given a tweet, classify whether one or more negative consequences are presented in the tweet. 

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a detrimental effect or penalty.
*   Illnesses in a SPECIFIC PERSON or a SPECIFIC GROUP OF PEOPLE, and future challenges facing a company or industry can also be considered negative consequences.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Negative Consequence: Mentions of adverse outcomes for one or more actors. E.g., damage from extreme weather or legal penalties faced by a corporation.

Examples:

a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a rising deforestation in the amazon, which can be seen as negative outcomes for the environment.

b) this will likely get little international media attention but it is a very very dark moment for press freedom in hk an apple daily columnist tweeted
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a very very dark moment for press freedom in hk, showing the negative consequence on the press in HK.

c) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned the sudden lift of the COVID restriction has negatively impacted the ability of private companies to protect their staffers and customers.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - negative consequence is not present.
Reasoning: No explicit mention of negative consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [43]:
test = pd.read_csv("test.csv")
test.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [ ]:
llm_results = []
llm_cots = []
model_to_use = "llama3.1" 

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet_text = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(tweet_text, model_name=model_to_use)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['negative_consequences_llm1'] = llm_results
test['negative_consequences_llm_cot1'] = llm_cots

In [46]:
llm_results = []
llm_cots = []
model_to_use = "llama3.1" 

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet_text = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(tweet_text, model_name=model_to_use)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['negative_consequences_llm2'] = llm_results
test['negative_consequences_llm_cot2'] = llm_cots

Classifying Tweets: 100%|██████████| 399/399 [11:54<00:00,  1.79s/it]


In [47]:
test.groupby(['negative_consequences_llm1','negative_consequences_llm2'])['topic'].count()

negative_consequences_llm1  negative_consequences_llm2
0                           0                             350
1                           1                              49
Name: topic, dtype: int64

In [48]:
krippendorff.alpha((test['negative_consequences_llm1'].values.tolist(), 
                    test['negative_consequences_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

In [49]:
test.groupby(['negative_consequences','negative_consequences_llm1'])['topic'].count()

negative_consequences  negative_consequences_llm1
0                      0                             348
                       1                               6
1                      0                               2
                       1                              43
Name: topic, dtype: int64

In [51]:
precision = precision_score(test['negative_consequences'], test['negative_consequences_llm1'])
recall = recall_score(test['negative_consequences'], test['negative_consequences_llm1'])
f1 = f1_score(test['negative_consequences'], test['negative_consequences_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 0.88
Recall: 0.96
F1-score: 0.91


In [52]:
test.to_csv('test.csv')

#### Make predictions for the full dataset

In [60]:
nf = pd.read_csv("newsfactor.csv")
nf.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [62]:
MESSAGE = """Given a tweet, classify whether one or more negative consequences are presented in the tweet. 

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a detrimental effect or penalty.
*   Illnesses in a SPECIFIC PERSON or a SPECIFIC GROUP OF PEOPLE, and future challenges facing a company or industry can also be considered negative consequences.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Negative Consequence: Mentions of adverse outcomes for one or more actors. E.g., damage from extreme weather or legal penalties faced by a corporation.

Examples:

a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a rising deforestation in the amazon, which can be seen as negative outcomes for the environment.

b) this will likely get little international media attention but it is a very very dark moment for press freedom in hk an apple daily columnist tweeted
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned a very very dark moment for press freedom in hk, showing the negative consequence on the press in HK.

c) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - negative consequence is present.
Reasoning: The tweet explicitly mentioned the sudden lift of the COVID restriction has negatively impacted the ability of private companies to protect their staffers and customers.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - negative consequence is not present.
Reasoning: No explicit mention of negative consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [63]:
llm_results = []
llm_cots = []
model_to_use = "llama3.1" 

for index, row in tqdm(nf.iterrows(), total=len(nf), desc="Classifying Tweets"):
    tweet_text = row['text_clean']
    llm_classification, llm_cot = classify_tweet_with_ollama(tweet_text, model_name=model_to_use)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

nf['negative_consequences'] = llm_results
nf['negative_consequences_cot'] = llm_cots

Classifying Tweets: 100%|██████████| 9610/9610 [4:51:19<00:00,  1.82s/it]  


In [64]:
nf.to_csv('newsfactor.csv')

### Positive consequences

#### Prompt instruction model

In [8]:
print("Locally available Ollama models:")
print(ollama.list())

Locally available Ollama models:
models=[Model(model='llama3.1:latest', modified_at=datetime.datetime(2025, 7, 22, 16, 29, 12, 748502, tzinfo=TzInfo(+02:00)), digest='46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e', size=4920753328, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_K_M')), Model(model='deepseek-r1:latest', modified_at=datetime.datetime(2025, 7, 22, 14, 42, 30, 846205, tzinfo=TzInfo(+02:00)), digest='6995872bfe4c521a67b32da386cd21d5c6e819b6e0d62f79f64ec83be99f5763', size=5225376047, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='8.2B', quantization_level='Q4_K_M'))]


In [5]:
messages = [
    {'role': 'user', 'content': """You are a highly precise prompt engineer. Trying to automatize part of the manual coding.
    Your goal is to generate a prompt that instructs an LLM to classify tweets.
    Below is a prompt you wrote for the varialbe negative consequences. Now you need to adapt this prompt for the variable positive consequences.
    
     Prompt:   
        Given a tweet, classify whether one or more negative consequences are presented in the tweet. 

        Note that:
        *   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a detrimental effect or penalty.
        *   Illnesses in a SPECIFIC PERSON or a SPECIFIC GROUP OF PEOPLE, and future challenges facing a company or industry can also be considered negative consequences.
        *   Do not infer, imply, or assume consequence if not explicitly stated.
        *   Do not introduce information that is not present in the text.

        Definition:
        Negative Consequence: Mentions of adverse outcomes for one or more actors. E.g., damage from extreme weather or legal penalties faced by a corporation.

        Examples:

        a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
        Answer: 1 - negative consequence is present.
        Reasoning: The tweet explicitly mentioned a rising deforestation in the amazon, which can be seen as negative outcomes for the environment.

        b) this will likely get little international media attention but it is a very very dark moment for press freedom in hk an apple daily columnist tweeted
        Answer: 1 - negative consequence is present.
        Reasoning: The tweet explicitly mentioned a very very dark moment for press freedom in hk, showing the negative consequence on the press in HK.

        c) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
        Answer: 1 - negative consequence is present.
        Reasoning: The tweet explicitly mentioned the sudden lift of the COVID restriction has negatively impacted the ability of private companies to protect their staffers and customers.

        d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
        Answer: 0 - negative consequence is not present.
        Reasoning: No explicit mention of negative consequence.

        THE ONLY OUTPUT should be a JSON object with two keys:
        1. `classification`: an integer (0 or 1) representing the classification result.
        2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

        Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

        Input tweet:
        '{tweet_text}'
    
    Here's the definiation of the variable positive consequences: Positive outcomes for one or more actors explicitly mentioned. E.g., Corporate success or consumer gains.
    Here're the examples for few shot learning:
        a) by building underground transmissions we are building a system that promotes clean and reliable energy learn more about this project in the latest episode of our plugged in podcast
        Answer: 1 - positive consequence is present.
        Reasoning: The tweet explicitly menionted the system could promote clean and reliable energy, which can be seen as positive outcomes of the system.
        b) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
        Answer: 1 - positive consequence is present.
        Reasoning: The tweet explicitly mentioned the strategies that can efficiently manage taxes in retirement, showing the positive consequence of the strategy.
        c) us and allied warfighters can explore multiple defensive plans in a matter of seconds with the command control battle management and communications system learn how c2bmc informs all levels of decision makers from the president to combatant commanders
        Answer: 1 - positive consequence is present.
        Reasoning: The tweet explicitly mentioned the system could inform decision makers in a fast and efficient way, which shows the positive outcome of the system.
        d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
        Answer: 0 - positive consequence is not present.
        Reasoning: No explicit mention of positive consequence.
        e) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
        Answer: 0 - positive consequence is not present.
        Reasoning: No explicit mention of positive consequence.
        f) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
        Answer: 0 - positive consequence is not present.
        Reasoning: No explicit mention of positive consequence.
    
    Always provide your chain of thought for generating the prompts but keep it short.
"""},
]

response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response:")
print(response['message']['content'])


Model Response:
Here's the adapted prompt for classifying variable positive consequences:

```
Given a tweet, classify whether one or more positive consequences are presented in the tweet.

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a beneficial effect or gain.
*   Corporate success, consumer gains, and other favorable outcomes can be considered positive consequences.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Positive Consequence: Mentions of favorable outcomes for one or more actors. E.g., Corporate success or consumer gains.

Examples:

a) by building underground transmissions we are building a system that promotes clean and reliable energy learn more about this project in the latest episode of our plugged in podcast
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the 

#### Try the prompt

In [6]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more positive consequences are presented in the tweet.

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a beneficial effect or gain.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Positive Consequence: Mentions of favorable outcomes for one or more actors. E.g., Corporate success or consumer gains.

Examples:

a) by building underground transmissions we are building a system that promotes clean and reliable energy learn more about this project in the latest episode of our plugged in podcast
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could promote clean and reliable energy, which can be seen as positive outcomes of the system.

b) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the strategies that can efficiently manage taxes in retirement, showing the positive consequence of the strategy.

c) us and allied warfighters can explore multiple defensive plans in a matter of seconds with the command control battle management and communications system learn how c2bmc informs all levels of decision makers from the president to combatant commanders
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could inform decision makers in a fast and efficient way, which shows the positive outcome of the system.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - positive consequence is not present.
Reasoning: No explicit mention of positive consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

Input tweet:
once you have talent offer upskilling or right skilling to help employees get better at their job or for the job they want to do pittorenatalie chief of enduring security framework national security agency womenatwork2021
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 1,
  "cot": "The tweet explicitly mentioned that upskilling or right skilling can help employees get better at their job or for the job they want to do, showing a positive consequence of offering these opportunities."
}


In [7]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more positive consequences are presented in the tweet.

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a beneficial effect or gain.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Positive Consequence: Mentions of favorable outcomes for one or more actors. E.g., Corporate success or consumer gains.

Examples:

a) by building underground transmissions we are building a system that promotes clean and reliable energy learn more about this project in the latest episode of our plugged in podcast
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could promote clean and reliable energy, which can be seen as positive outcomes of the system.

b) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the strategies that can efficiently manage taxes in retirement, showing the positive consequence of the strategy.

c) us and allied warfighters can explore multiple defensive plans in a matter of seconds with the command control battle management and communications system learn how c2bmc informs all levels of decision makers from the president to combatant commanders
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could inform decision makers in a fast and efficient way, which shows the positive outcome of the system.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - positive consequence is not present.
Reasoning: No explicit mention of positive consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

Input tweet:
in focus heimana reynolds for heimana reynolds the skateboard and park become ultimate platforms of creative and athletic expression watch heimana’s film from our exclusive documentary series now on twitter rlxteamusa teamusa tokyoolympics tokyo2020 olympics
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
    "classification": 1,
    "cot": "The tweet explicitly mentioned the skateboard and park can become ultimate platforms of creative and athletic expression, which can be seen as positive outcomes for Heimana Reynolds."
}


In [8]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more positive consequences are presented in the tweet.

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a beneficial effect or gain.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Positive Consequence: Mentions of favorable outcomes for one or more actors. E.g., Corporate success or consumer gains.

Examples:

a) by building underground transmissions we are building a system that promotes clean and reliable energy learn more about this project in the latest episode of our plugged in podcast
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could promote clean and reliable energy, which can be seen as positive outcomes of the system.

b) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the strategies that can efficiently manage taxes in retirement, showing the positive consequence of the strategy.

c) us and allied warfighters can explore multiple defensive plans in a matter of seconds with the command control battle management and communications system learn how c2bmc informs all levels of decision makers from the president to combatant commanders
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could inform decision makers in a fast and efficient way, which shows the positive outcome of the system.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - positive consequence is not present.
Reasoning: No explicit mention of positive consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

Input tweet:
the online trading app robinhood promised to wrest the stock market away from wall street’s gatekeepers — making it as easy to put millions at risk as it is to summon an uber it turns out robinhood was at the mercy of the industry it had vowed to upend
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 0,
  "cot": "No explicit mention of positive consequence."
}


In [9]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether one or more positive consequences are presented in the tweet.

Note that:
*   Only explicit mentions of outcomes or events are considered. The outcomes and events should have a beneficial effect or gain.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Positive Consequence: Mentions of favorable outcomes for one or more actors. E.g., Corporate success or consumer gains.

Examples:

a) by building underground transmissions we are building a system that promotes clean and reliable energy learn more about this project in the latest episode of our plugged in podcast
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could promote clean and reliable energy, which can be seen as positive outcomes of the system.

b) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the strategies that can efficiently manage taxes in retirement, showing the positive consequence of the strategy.

c) us and allied warfighters can explore multiple defensive plans in a matter of seconds with the command control battle management and communications system learn how c2bmc informs all levels of decision makers from the president to combatant commanders
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could inform decision makers in a fast and efficient way, which shows the positive outcome of the system.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - positive consequence is not present.
Reasoning: No explicit mention of positive consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences in your response. Output ONLY the JSON object.

Input tweet:
hundreds of netflix employees and suppoers are expected to take pa in a demonstration on wednesday to protest the streaming giants handling of the controversy surrounding dave chappelles recent comedy special the closer
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 0,
  "cot": "No explicit mention of positive consequence, the tweet only mentions protests and controversy."
}


#### Validate the performance

In [2]:
from ollama_classification import classify_tweet_with_ollama

In [51]:
MESSAGE = """Given a tweet, classify whether one or more positive consequences are presented in the tweet.

Note that:
*   Only explicit mentions of outcomes or events that have a beneficial effect or gain are considered.
*   Do not classify positive consequences based solely on positive words or general sentiment: E.g., "happy birthday," "wish you a great day." These are not positive consequences.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Positive Consequences: Mentions of favorable outcomes for one or more actors. E.g., Corporate success, consumer's needs are fulfilled, someone has won an award.

Examples:

a) by building underground transmissions we are building a system that promotes clean and reliable energy learn more about this project in the latest episode of our plugged in podcast
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could promote clean and reliable energy, which can be seen as positive outcomes of the system.

b) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the strategies that can efficiently manage taxes in retirement, showing the positive consequence of the strategy.

c) us and allied warfighters can explore multiple defensive plans in a matter of seconds with the command control battle management and communications system learn how c2bmc informs all levels of decision makers from the president to combatant commanders
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could inform decision makers in a fast and efficient way, which shows the positive outcome of the system.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - positive consequence is not present.
Reasoning: No explicit mention of positive consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [52]:
val = pd.read_csv("val.csv")
val.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [53]:
llm_results = []
llm_cots = []

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['positive_consequences_llm1'] = llm_results
val['positive_consequences_llm_cot1'] = llm_cots

Classifying Tweets: 100%|██████████| 81/81 [03:12<00:00,  2.38s/it]


In [58]:
llm_results = []
llm_cots = []

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['positive_consequences_llm2'] = llm_results
val['positive_consequences_llm_cot2'] = llm_cots

Classifying Tweets: 100%|██████████| 81/81 [03:09<00:00,  2.35s/it]


In [59]:
val.groupby(['positive_consequences_llm1','positive_consequences_llm2'])['topic'].count()

positive_consequences_llm1  positive_consequences_llm2
0                           0                             43
1                           1                             38
Name: topic, dtype: int64

In [60]:
krippendorff.alpha((val['positive_consequences_llm1'].values.tolist(), 
                    val['positive_consequences_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

In [61]:
val.groupby(['positive_consequences','positive_consequences_llm1'])['topic'].count()

positive_consequences  positive_consequences_llm1
0                      0                             37
                       1                              8
1                      0                              6
                       1                             30
Name: topic, dtype: int64

In [62]:
precision = precision_score(val['positive_consequences'], val['positive_consequences_llm1'])
recall = recall_score(val['positive_consequences'], val['positive_consequences_llm1'])
f1 = f1_score(val['positive_consequences'], val['positive_consequences_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 0.79
Recall: 0.83
F1-score: 0.81


In [63]:
val.to_csv('val.csv')

#### Test the final prompt

In [ ]:
MESSAGE = """Given a tweet, classify whether one or more positive consequences are presented in the tweet.

Note that:
*   Only explicit mentions of outcomes or events that have a beneficial effect or gain are considered.
*   Do not classify positive consequences based solely on positive words or general sentiment: E.g., "happy birthday," "wish you a great day." These are not positive consequences.
*   Do not infer, imply, or assume consequence if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Positive Consequences: Mentions of favorable outcomes for one or more actors. E.g., Corporate success, consumer's needs are fulfilled, someone has won an award.

Examples:

a) by building underground transmissions we are building a system that promotes clean and reliable energy learn more about this project in the latest episode of our plugged in podcast
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could promote clean and reliable energy, which can be seen as positive outcomes of the system.

b) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the strategies that can efficiently manage taxes in retirement, showing the positive consequence of the strategy.

c) us and allied warfighters can explore multiple defensive plans in a matter of seconds with the command control battle management and communications system learn how c2bmc informs all levels of decision makers from the president to combatant commanders
Answer: 1 - positive consequence is present.
Reasoning: The tweet explicitly mentioned the system could inform decision makers in a fast and efficient way, which shows the positive outcome of the system.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - positive consequence is not present.
Reasoning: No explicit mention of positive consequence.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [64]:
test = pd.read_csv("test.csv")
test.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [ ]:
llm_results = []
llm_cots = []

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['positive_consequences_llm1'] = llm_results
test['positive_consequences_llm_cot1'] = llm_cots

In [ ]:
llm_results = []
llm_cots = []

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['positive_consequences_llm2'] = llm_results
test['positive_consequences_llm_cot2'] = llm_cots

In [65]:
test.groupby(['positive_consequences_llm1','positive_consequences_llm2'])['topic'].count()

positive_consequences_llm1  positive_consequences_llm2
0                           0                             218
1                           1                             181
Name: topic, dtype: int64

In [66]:
krippendorff.alpha((test['positive_consequences_llm1'].values.tolist(), 
                    test['positive_consequences_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

In [67]:
test.groupby(['positive_consequences','positive_consequences_llm1'])['topic'].count()

positive_consequences  positive_consequences_llm1
0                      0                             202
                       1                              45
1                      0                              16
                       1                             136
Name: topic, dtype: int64

In [68]:
precision = precision_score(test['positive_consequences'], test['positive_consequences_llm1'])
recall = recall_score(test['positive_consequences'], test['positive_consequences_llm1'])
f1 = f1_score(test['positive_consequences'], test['positive_consequences_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 0.75
Recall: 0.89
F1-score: 0.82


In [ ]:
test.to_csv('test.csv')

#### Make predictions for the full dataset

In [69]:
nf = pd.read_csv("newsfactor.csv")
nf.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [71]:
llm_results = []
llm_cots = []

for index, row in tqdm(nf.iterrows(), total=len(nf), desc="Classifying Tweets"):
    tweet = row['text_clean']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

nf['positive_consequences_llm'] = llm_results
nf['positive_consequences_llm_cot'] = llm_cots

Classifying Tweets:  99%|█████████▉| 9498/9610 [6:01:45<05:11,  2.78s/it]  

LLM raw output: {
  "classification": 1,
  "cot": "The tweet explicitly mentioned that PG\&E is better positioned to...


Classifying Tweets: 100%|██████████| 9610/9610 [6:06:28<00:00,  2.29s/it]


In [79]:
nf.to_csv('newsfactor.csv')

### Scope

#### Prompt instruction model

In [80]:
print("Locally available Ollama models:")
print(ollama.list())

Locally available Ollama models:
models=[Model(model='llama3.1:latest', modified_at=datetime.datetime(2025, 7, 22, 16, 29, 12, 748502, tzinfo=TzInfo(+02:00)), digest='46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e', size=4920753328, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_K_M')), Model(model='deepseek-r1:latest', modified_at=datetime.datetime(2025, 7, 22, 14, 42, 30, 846205, tzinfo=TzInfo(+02:00)), digest='6995872bfe4c521a67b32da386cd21d5c6e819b6e0d62f79f64ec83be99f5763', size=5225376047, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='8.2B', quantization_level='Q4_K_M'))]


In [81]:
messages = [
    {'role': 'user', 'content': """You are a highly precise prompt engineer. Trying to automatize part of the manual coding.
    Your goal is to generate a prompt that instructs an LLM to classify tweets.
    Below is a prompt you wrote for the varialbe positive consequences. Now you need to adapt this prompt for the variable scope.
    
     Prompt:   
        Given a tweet, classify whether one or more positive consequences are presented in the tweet.

        Note that:
        *   Only explicit mentions of outcomes or events that have a beneficial effect or gain are considered.
        *   Do not classify positive consequences based solely on positive words or general sentiment: E.g., "happy birthday," "wish you a great day." These are not positive consequences.
        *   Do not infer, imply, or assume consequence if not explicitly stated.
        *   Do not introduce information that is not present in the text.

        Definition:
        Positive Consequences: Mentions of favorable outcomes for one or more actors. E.g., Corporate success, consumer's needs are fulfilled, someone has won an award.

        Examples:

        a) by building underground transmissions we are building a system that promotes clean and reliable energy learn more about this project in the latest episode of our plugged in podcast
        Answer: 1 - positive consequence is present.
        Reasoning: The tweet explicitly mentioned the system could promote clean and reliable energy, which can be seen as positive outcomes of the system.

        b) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
        Answer: 1 - positive consequence is present.
        Reasoning: The tweet explicitly mentioned the strategies that can efficiently manage taxes in retirement, showing the positive consequence of the strategy.

        c) us and allied warfighters can explore multiple defensive plans in a matter of seconds with the command control battle management and communications system learn how c2bmc informs all levels of decision makers from the president to combatant commanders
        Answer: 1 - positive consequence is present.
        Reasoning: The tweet explicitly mentioned the system could inform decision makers in a fast and efficient way, which shows the positive outcome of the system.

        d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
        Answer: 0 - positive consequence is not present.
        Reasoning: No explicit mention of positive consequence.

        THE ONLY OUTPUT should be a JSON object with two keys:
        1. `classification`: an integer (0 or 1) representing the classification result.
        2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

        Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

        Input tweet:
        '{tweet_text}'
    
    Here's the definiation of the variable scope: Scope refers to the number of people, organizations, or regions involved or affected. Does the tweet explicitly refer to the relevance of a topic on individuals, companies, regions, or countries?
    Note. This includes both literal numbers mentioned with a verifiable source, e.g., “thousands of people”, or a single person or company, as well as figurative speeches of a group of people or company, such as “all Netflix-viewers”, “Every woman living in Germany”, “Germany”, or "the whole smart phone industry". 
    Here're the examples for few shot learning:
        a) were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other food justice work at bhem
        Answer: 1 - scope is present.
        Reasoning: The tweet explicitly menionted "black communities" and "a hungerfree world for all" as figerative scope.
        b) why the netflix hit squid game is resonating around the world
        Answer: 1 - scope is present.
        Reasoning: The tweet explicitly mentioned "around the world" as a figerative scope.
        c) a recent survey by the brennan center for justice found 1 in 3 election officials nationwide feels unsafe at work nearly 1 in 5 called threats to their lives a jobrelated concern
        Answer: 1 - scope is present.
        Reasoning: The tweet explicitly mentioned "1 in 3 election officials nationwide" and "1 in 5", showing scope with literal numbers.
        d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
        Answer: 0 - scope is not present.
        Reasoning: No explicit mention of scope. Just mentioning "you" without explicitly show how the person is involved or affected by anything
        e) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
        Answer: 0 - scope is not present.
        Reasoning: Although a person "ceo dr lisa su" is mentioned, the tweet did not explicitly show she is involved or affected by anything.
        f) moisturize skin while you bathe skinsosoft
        Answer: 0 - scope is not present.
        Reasoning: No explicit mention of scope.
    
    Always provide your chain of thought for generating the prompts but keep it short.
"""},
]

response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response:")
print(response['message']['content'])


Model Response:
**Prompt for Variable Scope**

Here's the adapted prompt for variable scope:

Given a tweet, classify whether one or more scopes are presented in the tweet.

Note that:
*   Only explicit mentions of relevance or impact on individuals, companies, regions, or countries are considered.
*   Do not infer, imply, or assume scope if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Scope: Explicit references to the number of people, organizations, or regions involved or affected. This includes literal numbers mentioned with a verifiable source (e.g., “thousands of people”) as well as figurative speeches of a group of people or company (e.g., “all Netflix-viewers”, “Every woman living in Germany”, etc.).

Examples:

a) were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other f

#### Try the prompt

In [83]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether the scope of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of relevance or impact on individuals, companies, regions, or countries are considered.
*   Do not infer, imply, or assume scope if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Scope: Explicit references to the number of people, organizations, or regions involved or affected. This includes literal numbers mentioned with a verifiable source (e.g., “thousands of people”) as well as figurative speeches of a group of people or company (e.g., “all Netflix-viewers”, “Every woman living in Germany”, etc.).

Examples:

a) were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other food justice work at bhem
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "black communities" as a figurative scope.

b) why the netflix hit squid game is resonating around the world
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "around the world" as a figurative scope.

c) a recent survey by the brennan center for justice found 1 in 3 election officials nationwide feels unsafe at work nearly 1 in 5 called threats to their lives a jobrelated concern
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "1 in 3 election officials nationwide" and "1 in 5", showing scope with literal numbers.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - scope is not present.
Reasoning: No explicit mention of scope. Just mentioning "you" without explicitly show how the person is involved or affected by anything

e) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
Answer: 0 - scope is not present.
Reasoning: Although a person "ceo dr lisa su" is mentioned, the tweet did not explicitly show she is involved or affected by anything.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ``json or ```) in your response. Output ONLY the JSON object.

Input tweet:
why are customers switching to our cloud applications stevenrmiranda discusses with leaders from kroger mdlz and ttxcompany at oraclelive
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 0,
  "cot": "No explicit mention of scope. Mentioning 'our' could be seen as a figurative speech, but it doesn't explicitly show customers are involved or affected by anything."
}


In [84]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether the scope of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of relevance or impact on individuals, companies, regions, or countries are considered.
*   Do not infer, imply, or assume scope if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Scope: Explicit references to the number of people, organizations, or regions involved or affected. This includes literal numbers mentioned with a verifiable source (e.g., “thousands of people”) as well as figurative speeches of a group of people or company (e.g., “all Netflix-viewers”, “Every woman living in Germany”, etc.).

Examples:

a) were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other food justice work at bhem
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "black communities" as a figurative scope.

b) why the netflix hit squid game is resonating around the world
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "around the world" as a figurative scope.

c) a recent survey by the brennan center for justice found 1 in 3 election officials nationwide feels unsafe at work nearly 1 in 5 called threats to their lives a jobrelated concern
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "1 in 3 election officials nationwide" and "1 in 5", showing scope with literal numbers.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - scope is not present.
Reasoning: No explicit mention of scope. Just mentioning "you" without explicitly show how the person is involved or affected by anything

e) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
Answer: 0 - scope is not present.
Reasoning: Although a person "ceo dr lisa su" is mentioned, the tweet did not explicitly show she is involved or affected by anything.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ``json or ```) in your response. Output ONLY the JSON object.

Input tweet:
the newest crop of iot devices to take up residence in your home those that put healthcare in your hands   heres why careostech fluolabsllc medwand miiskinapp cologuard
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
```json
{
  "classification": 0,
  "cot": "No explicit mention of scope. The tweet only mentions 'your' without explicitly showing how the person is involved or affected by anything"
}
```


In [85]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether the scope of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of relevance or impact on individuals, companies, regions, or countries are considered.
*   Do not infer, imply, or assume scope if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Scope: Explicit references to the number of people, organizations, or regions involved or affected. This includes literal numbers mentioned with a verifiable source (e.g., “thousands of people”) as well as figurative speeches of a group of people or company (e.g., “all Netflix-viewers”, “Every woman living in Germany”, etc.).

Examples:

a) were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other food justice work at bhem
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "black communities" as a figurative scope.

b) why the netflix hit squid game is resonating around the world
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "around the world" as a figurative scope.

c) a recent survey by the brennan center for justice found 1 in 3 election officials nationwide feels unsafe at work nearly 1 in 5 called threats to their lives a jobrelated concern
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "1 in 3 election officials nationwide" and "1 in 5", showing scope with literal numbers.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - scope is not present.
Reasoning: No explicit mention of scope. Just mentioning "you" without explicitly show how the person is involved or affected by anything

e) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
Answer: 0 - scope is not present.
Reasoning: Although a person "ceo dr lisa su" is mentioned, the tweet did not explicitly show she is involved or affected by anything.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ``json or ```) in your response. Output ONLY the JSON object.

Input tweet:
president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 1,
  "cot": "The tweet explicitly mentioned 'brazil' and implicitly, the international community which implies billions of dollars, showing scope with literal numbers."
}


In [86]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether the scope of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of relevance or impact on individuals, companies, regions, or countries are considered.
*   Do not infer, imply, or assume scope if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Scope: Explicit references to the number of people, organizations, or regions involved or affected. This includes literal numbers mentioned with a verifiable source (e.g., “thousands of people”) as well as figurative speeches of a group of people or company (e.g., “all Netflix-viewers”, “Every woman living in Germany”, etc.).

Examples:

a) were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other food justice work at bhem
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "black communities" as a figurative scope.

b) why the netflix hit squid game is resonating around the world
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "around the world" as a figurative scope.

c) a recent survey by the brennan center for justice found 1 in 3 election officials nationwide feels unsafe at work nearly 1 in 5 called threats to their lives a jobrelated concern
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "1 in 3 election officials nationwide" and "1 in 5", showing scope with literal numbers.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - scope is not present.
Reasoning: No explicit mention of scope. Just mentioning "you" without explicitly show how the person is involved or affected by anything

e) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
Answer: 0 - scope is not present.
Reasoning: Although a person "ceo dr lisa su" is mentioned, the tweet did not explicitly show she is involved or affected by anything.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ``json or ```) in your response. Output ONLY the JSON object.

Input tweet:
hundreds of netflix employees and suppoers are expected to take pa in a demonstration on wednesday to protest the streaming giants handling of the controversy surrounding dave chappelles recent comedy special the closer
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 1,
  "cot": "The tweet explicitly mentioned 'hundreds of netflix employees and supporters' as a literal number, showing scope."
}


#### Validate the performance

In [87]:
from ollama_classification import classify_tweet_with_ollama

In [107]:
MESSAGE = """Given a tweet, classify whether the scope of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of relevance or impact on individuals, companies, regions, or countries are considered. The time range of an issue or event, e.g., "the future of", "the month of", "a week", are not scope.
*   Do not infer, imply, or assume scope if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Scope: Explicit references to the number of people, organizations, or regions involved or affected. This includes literal numbers mentioned with a verifiable source (e.g., “thousands of people”) as well as figurative speeches of a group of people or company (e.g., “all Netflix-viewers”, “Every woman living in Germany”, etc.).

Examples:

a) were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other food justice work at bhem
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "black communities" as a figurative scope of the food justice issue.

b) why the netflix hit squid game is resonating around the world
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "resonating around the world" as a figurative scope of squid game.

c) a recent survey by the brennan center for justice found 1 in 3 election officials nationwide feels unsafe at work nearly 1 in 5 called threats to their lives a jobrelated concern
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "1 in 3 election officials nationwide" and "1 in 5", showing scope with literal numbers.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - scope is not present.
Reasoning: No explicit mention of scope. Just mentioning "you" without explicitly show how the person is involved or affected by anything

e) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
Answer: 0 - scope is not present.
Reasoning: Although a person "ceo dr lisa su" is mentioned, the tweet did not explicitly show she is involved or affected by anything.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [113]:
val = pd.read_csv("val.csv")
val.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [109]:
llm_results = []
llm_cots = []

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['scope_llm1'] = llm_results
val['scope_llm_cot1'] = llm_cots

Classifying Tweets: 100%|██████████| 81/81 [03:49<00:00,  2.83s/it]


In [116]:
llm_results = []
llm_cots = []

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['scope_llm2'] = llm_results
val['scope_llm_cot2'] = llm_cots

Classifying Tweets: 100%|██████████| 81/81 [03:46<00:00,  2.80s/it]


In [117]:
val.groupby(['scope_llm1','scope_llm2'])['topic'].count()

scope_llm1  scope_llm2
0           0             51
1           1             30
Name: topic, dtype: int64

In [118]:
krippendorff.alpha((val['scope_llm1'].values.tolist(), 
                    val['scope_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

In [119]:
val.groupby(['scope','scope_llm1'])['topic'].count()

scope  scope_llm1
0      0             45
       1              4
1      0              6
       1             26
Name: topic, dtype: int64

In [120]:
precision = precision_score(val['scope'], val['scope_llm1'])
recall = recall_score(val['scope'], val['scope_llm1'])
f1 = f1_score(val['scope'], val['scope_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 0.87
Recall: 0.81
F1-score: 0.84


In [121]:
val.to_csv('val.csv')

#### Test the final prompt

In [ ]:
MESSAGE = """Given a tweet, classify whether the scope of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of relevance or impact on individuals, companies, regions, or countries are considered. The time range of an issue or event, e.g., "the future of", "the month of", "a week", are not scope.
*   Do not infer, imply, or assume scope if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Scope: Explicit references to the number of people, organizations, or regions involved or affected. This includes literal numbers mentioned with a verifiable source (e.g., “thousands of people”) as well as figurative speeches of a group of people or company (e.g., “all Netflix-viewers”, “Every woman living in Germany”, etc.).

Examples:

a) were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other food justice work at bhem
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "black communities" as a figurative scope of the food justice issue.

b) why the netflix hit squid game is resonating around the world
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "resonating around the world" as a figurative scope of squid game.

c) a recent survey by the brennan center for justice found 1 in 3 election officials nationwide feels unsafe at work nearly 1 in 5 called threats to their lives a jobrelated concern
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "1 in 3 election officials nationwide" and "1 in 5", showing scope with literal numbers.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - scope is not present.
Reasoning: No explicit mention of scope. Just mentioning "you" without explicitly show how the person is involved or affected by anything

e) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
Answer: 0 - scope is not present.
Reasoning: Although a person "ceo dr lisa su" is mentioned, the tweet did not explicitly show she is involved or affected by anything.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [122]:
test = pd.read_csv("test.csv")
test.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [ ]:
llm_results = []
llm_cots = []

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['scope_llm1'] = llm_results
test['scope_llm_cot1'] = llm_cots

In [ ]:
llm_results = []
llm_cots = []

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['scope_llm2'] = llm_results
test['scope_llm_cot2'] = llm_cots

In [123]:
test.groupby(['scope_llm1','scope_llm2'])['topic'].count()

scope_llm1  scope_llm2
0           0             235
1           1             164
Name: topic, dtype: int64

In [124]:
krippendorff.alpha((test['scope_llm1'].values.tolist(), 
                    test['scope_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

In [125]:
test.groupby(['scope','scope_llm1'])['topic'].count()

scope  scope_llm1
0      0             220
       1              30
1      0              15
       1             134
Name: topic, dtype: int64

In [126]:
precision = precision_score(test['scope'], test['scope_llm1'])
recall = recall_score(test['scope'], test['scope_llm1'])
f1 = f1_score(test['scope'], test['scope_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 0.82
Recall: 0.90
F1-score: 0.86


In [ ]:
test.to_csv('test.csv')

#### Make predictions for the full dataset

In [127]:
nf = pd.read_csv("newsfactor.csv")
nf.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [128]:
MESSAGE = """Given a tweet, classify whether the scope of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of relevance or impact on individuals, companies, regions, or countries are considered. The time range of an issue or event, e.g., "the future of", "the month of", "a week", are not scope.
*   Do not infer, imply, or assume scope if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Scope: Explicit references to the number of people, organizations, or regions involved or affected. This includes literal numbers mentioned with a verifiable source (e.g., “thousands of people”) as well as figurative speeches of a group of people or company (e.g., “all Netflix-viewers”, “Every woman living in Germany”, etc.).

Examples:

a) were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other food justice work at bhem
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "black communities" as a figurative scope of the food justice issue.

b) why the netflix hit squid game is resonating around the world
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "resonating around the world" as a figurative scope of squid game.

c) a recent survey by the brennan center for justice found 1 in 3 election officials nationwide feels unsafe at work nearly 1 in 5 called threats to their lives a jobrelated concern
Answer: 1 - scope is present.
Reasoning: The tweet explicitly mentioned "1 in 3 election officials nationwide" and "1 in 5", showing scope with literal numbers.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - scope is not present.
Reasoning: No explicit mention of scope. Just mentioning "you" without explicitly show how the person is involved or affected by anything

e) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
Answer: 0 - scope is not present.
Reasoning: Although a person "ceo dr lisa su" is mentioned, the tweet did not explicitly show she is involved or affected by anything.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [129]:
llm_results = []
llm_cots = []

for index, row in tqdm(nf.iterrows(), total=len(nf), desc="Classifying Tweets"):
    tweet = row['text_clean']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

nf['scope_llm'] = llm_results
nf['scope_llm_cot'] = llm_cots

Classifying Tweets:  24%|██▍       | 2334/9610 [1:48:47<6:17:50,  3.12s/it]

LLM raw output: {
    "classification": 1,
    "cot": "The tweet explicitly mentioned 'the "trump administration tar...


Classifying Tweets:  72%|███████▏  | 6933/9610 [5:23:24<2:00:21,  2.70s/it]

LLM raw output: {
  "classification": 1,
  "cot": "The tweet explicitly mentioned 'Kansas\' meatpacking workers', sh...


Classifying Tweets: 100%|██████████| 9610/9610 [7:26:43<00:00,  2.79s/it]  


In [130]:
nf.to_csv('newsfactor.csv')

### Controversy

#### Prompt instruction model

In [3]:
print("Locally available Ollama models:")
print(ollama.list())

Locally available Ollama models:
models=[Model(model='llama3.1:latest', modified_at=datetime.datetime(2025, 7, 22, 16, 29, 12, 748502, tzinfo=TzInfo(+02:00)), digest='46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e', size=4920753328, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_K_M')), Model(model='deepseek-r1:latest', modified_at=datetime.datetime(2025, 7, 22, 14, 42, 30, 846205, tzinfo=TzInfo(+02:00)), digest='6995872bfe4c521a67b32da386cd21d5c6e819b6e0d62f79f64ec83be99f5763', size=5225376047, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='8.2B', quantization_level='Q4_K_M'))]


In [4]:
messages = [
    {'role': 'user', 'content': """You are a highly precise prompt engineer. Trying to automatize part of the manual coding.
    Your goal is to generate a prompt that instructs an LLM to classify tweets.
    Below is a prompt you wrote for the varialbe scope. Now you need to adapt this prompt for the variable controversy.
    
     Prompt:   
        Given a tweet, classify whether the scope of an issue or event is presented in the tweet.

        Note that:
        *   Only explicit mentions of relevance or impact on individuals, companies, regions, or countries are considered. The time range of an issue or event, e.g., "the future of", "the month of", "a week", are not scope.
        *   Do not infer, imply, or assume scope if not explicitly stated.
        *   Do not introduce information that is not present in the text.

        Definition:
        Scope: Explicit references to the number of people, organizations, or regions involved or affected. This includes literal numbers mentioned with a verifiable source (e.g., “thousands of people”) as well as figurative speeches of a group of people or company (e.g., “all Netflix-viewers”, “Every woman living in Germany”, etc.).

        Examples:

        a) were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other food justice work at bhem
        Answer: 1 - scope is present.
        Reasoning: The tweet explicitly mentioned "black communities" as a figurative scope of the food justice issue.

        b) why the netflix hit squid game is resonating around the world
        Answer: 1 - scope is present.
        Reasoning: The tweet explicitly mentioned "resonating around the world" as a figurative scope of squid game.

        c) a recent survey by the brennan center for justice found 1 in 3 election officials nationwide feels unsafe at work nearly 1 in 5 called threats to their lives a jobrelated concern
        Answer: 1 - scope is present.
        Reasoning: The tweet explicitly mentioned "1 in 3 election officials nationwide" and "1 in 5", showing scope with literal numbers.

        d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
        Answer: 0 - scope is not present.
        Reasoning: No explicit mention of scope. Just mentioning "you" without explicitly show how the person is involved or affected by anything

        e) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
        Answer: 0 - scope is not present.
        Reasoning: Although a person "ceo dr lisa su" is mentioned, the tweet did not explicitly show she is involved or affected by anything.

        THE ONLY OUTPUT should be a JSON object with two keys:
        1. `classification`: an integer (0 or 1) representing the classification result.
        2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

        Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

        Input tweet:
        '{tweet_text}'
    
    Here's the definiation of the variable controversy: Controversy refers to the explicit presentation of opposing opinions, debates, or disputes. E.g., a debate about a lawsuit, disagreement with a policy.
    Note. This includes arguments, splits, strikes, fights, insurrections, and warfare. The key question to ask is: Is the focus of the story on the conflict or controversy? 
    Here're the examples for few shot learning:
        a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
        Answer: 1 - controversy is present.
        Reasoning: The tweet explicitly menionted the president jair bolsonaro wants the international community to pledge billions of dollars but the the donors are skeptical. These are two opposing opinions, which means controversy is present.
        b) hundreds of netflix employees and suppoers are expected to take pa in a demonstration on wednesday to protest the streaming giants handling of the controversy surrounding dave chappelles recent comedy special the closer
        Answer: 1 - controversy is present.
        Reasoning: The tweet explicitly mentioned employees are suppoters are taking part in a demonstration to protest against Netflix, which clearly shows the conflict.
        c) the online trading app robinhood promised to wrest the stock market away from wall street’s gatekeepers — making it as easy to put millions at risk as it is to summon an uber it turns out robinhood was at the mercy of the industry it had vowed to upend
        Answer: 1 - controversy is present.
        Reasoning: The tweet highlights Robinhood’s promise to challenge Wall Street's dominance but reveals its dependence on the very system it aimed to disrupt, clearly showing the conflict between its mission and reality.
        d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
        Answer: 0 - controversy is not present.
        Reasoning: No explicit mention of opposing opinions and disputes.
        e) hong kongs prodemocracy newspaper apple daily is shutting down after 26 years the move comes after the government froze its bank accounts and arrested several top executives including the editorinchief
        Answer: 0 - controversy is not present.
        Reasoning: No explicit mention of opposing opinions and disputes.
        f) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
        Answer: 0 - controversy is not present.
        Reasoning: No explicit mention of opposing opinions and disputes.
    
    Always provide your chain of thought for generating the prompts but keep it short.
"""},
]

response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response:")
print(response['message']['content'])


Model Response:
Here's the adapted prompt for the variable "controversy":

```
Given a tweet, classify whether the controversy of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of opposing opinions, debates, or disputes are considered. E.g., a debate about a lawsuit, disagreement with a policy.
*   Do not infer, imply, or assume controversy if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Controversy: Explicit presentation of opposing opinions, debates, or disputes. E.g., a debate about a lawsuit, disagreement with a policy.

Examples:

a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation efforts but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - controversy is present.
Reasoning: The tweet explicitly mentioned opposing opinions between the pres

#### Try the prompt

In [6]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether the controversy of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of opposing opinions, debates, or disputes are considered. E.g., a debate about a lawsuit, disagreement with a policy.
*   Do not infer, imply, or assume controversy if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Controversy: Explicit presentation of opposing opinions, debates, or disputes. E.g., a debate about a lawsuit, disagreement with a policy.

Examples:
a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation efforts but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - controversy is present.
Reasoning: The tweet explicitly mentioned opposing opinions between the president's request and the donors' skepticism.

b) hundreds of netflix employees and supporters are expected to take part in a demonstration on wednesday to protest the streaming giant's handling of the controversy surrounding dave chappelle's recent comedy special
Answer: 1 - controversy is present.
Reasoning: The tweet explicitly mentioned opposing opinions between Netflix and its employees/ supporters with the demonstration and protest.

c) the online trading app robinhood promised to wrest the stock market away from wall street’s gatekeepers — making it as easy to put millions at risk as it is to summon an uber it turns out robinhood was at the mercy of the industry it had vowed to upend
Answer: 1 - controversy is present.
Reasoning: The tweet highlights Robinhood’s promise to challenge Wall Street's dominance but reveals its dependence on the very system it aimed to disrupt, showing conflicts between Robinhood's mission and reality.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - controversy is not present.
Reasoning: No explicit mention of opposing opinions and disputes.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

DO NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
why are customers switching to our cloud applications stevenrmiranda discusses with leaders from kroger mdlz and ttxcompany at oraclelive
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 0,
  "cot": "No explicit mention of opposing opinions, debates, or disputes"
}


In [7]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether the controversy of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of opposing opinions, debates, or disputes are considered. E.g., a debate about a lawsuit, disagreement with a policy.
*   Do not infer, imply, or assume controversy if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Controversy: Explicit presentation of opposing opinions, debates, or disputes. E.g., a debate about a lawsuit, disagreement with a policy.

Examples:
a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation efforts but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - controversy is present.
Reasoning: The tweet explicitly mentioned opposing opinions between the president's request and the donors' skepticism.

b) hundreds of netflix employees and supporters are expected to take part in a demonstration on wednesday to protest the streaming giant's handling of the controversy surrounding dave chappelle's recent comedy special
Answer: 1 - controversy is present.
Reasoning: The tweet explicitly mentioned opposing opinions between Netflix and its employees/ supporters with the demonstration and protest.

c) the online trading app robinhood promised to wrest the stock market away from wall street’s gatekeepers — making it as easy to put millions at risk as it is to summon an uber it turns out robinhood was at the mercy of the industry it had vowed to upend
Answer: 1 - controversy is present.
Reasoning: The tweet highlights Robinhood’s promise to challenge Wall Street's dominance but reveals its dependence on the very system it aimed to disrupt, showing conflicts between Robinhood's mission and reality.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - controversy is not present.
Reasoning: No explicit mention of opposing opinions and disputes.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
```json
{
  "classification": 1,
  "cot": "The tweet explicitly mentioned a disagreement between former Fed Vice Chairman Donald Kohn and the US Central Bank regarding their preparedness for faster inflation."
}
```


#### Validate the performance

In [8]:
from ollama_classification import classify_tweet_with_ollama

In [9]:
MESSAGE = """Given a tweet, classify whether the controversy of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of opposing opinions, debates, or disputes are considered. E.g., a debate about a lawsuit, disagreement with a policy.
*   Do not infer, imply, or assume controversy if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Controversy: Explicit presentation of opposing opinions, debates, or disputes. E.g., a debate about a lawsuit, disagreement with a policy.

Examples:
a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation efforts but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - controversy is present.
Reasoning: The tweet explicitly mentioned opposing opinions between the president's request and the donors' skepticism.

b) hundreds of netflix employees and supporters are expected to take part in a demonstration on wednesday to protest the streaming giant's handling of the controversy surrounding dave chappelle's recent comedy special
Answer: 1 - controversy is present.
Reasoning: The tweet explicitly mentioned opposing opinions between Netflix and its employees/ supporters with the demonstration and protest.

c) the online trading app robinhood promised to wrest the stock market away from wall street’s gatekeepers — making it as easy to put millions at risk as it is to summon an uber it turns out robinhood was at the mercy of the industry it had vowed to upend
Answer: 1 - controversy is present.
Reasoning: The tweet highlights Robinhood’s promise to challenge Wall Street's dominance but reveals its dependence on the very system it aimed to disrupt, showing conflicts between Robinhood's mission and reality.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - controversy is not present.
Reasoning: No explicit mention of opposing opinions and disputes.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [15]:
val = pd.read_csv("val.csv")
val.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [11]:
llm_results = []
llm_cots = []

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['controversy_llm1'] = llm_results
val['controversy_llm_cot1'] = llm_cots

Classifying Tweets: 100%|██████████| 81/81 [02:36<00:00,  1.93s/it]


In [18]:
llm_results = []
llm_cots = []

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['controversy_llm2'] = llm_results
val['controversy_llm_cot2'] = llm_cots

Classifying Tweets: 100%|██████████| 81/81 [02:42<00:00,  2.00s/it]


In [19]:
val.groupby(['controversy_llm1','controversy_llm2'])['topic'].count()

controversy_llm1  controversy_llm2
0                 0                   73
1                 1                    8
Name: topic, dtype: int64

In [20]:
krippendorff.alpha((val['controversy_llm1'].values.tolist(), 
                    val['controversy_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

In [21]:
val.groupby(['controversy','controversy_llm1'])['topic'].count()

controversy  controversy_llm1
0            0                   72
             1                    2
1            0                    1
             1                    6
Name: topic, dtype: int64

In [22]:
precision = precision_score(val['controversy'], val['controversy_llm1'])
recall = recall_score(val['controversy'], val['controversy_llm1'])
f1 = f1_score(val['controversy'], val['controversy_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 0.75
Recall: 0.86
F1-score: 0.80


In [23]:
val.to_csv('val.csv')

#### Test the final prompt

In [24]:
MESSAGE = """Given a tweet, classify whether the controversy of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of opposing opinions, debates, or disputes are considered. E.g., a debate about a lawsuit, disagreement with a policy.
*   Do not infer, imply, or assume controversy if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Controversy: Explicit presentation of opposing opinions, debates, or disputes. E.g., a debate about a lawsuit, disagreement with a policy.

Examples:
a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation efforts but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - controversy is present.
Reasoning: The tweet explicitly mentioned opposing opinions between the president's request and the donors' skepticism.

b) hundreds of netflix employees and supporters are expected to take part in a demonstration on wednesday to protest the streaming giant's handling of the controversy surrounding dave chappelle's recent comedy special
Answer: 1 - controversy is present.
Reasoning: The tweet explicitly mentioned opposing opinions between Netflix and its employees/ supporters with the demonstration and protest.

c) the online trading app robinhood promised to wrest the stock market away from wall street’s gatekeepers — making it as easy to put millions at risk as it is to summon an uber it turns out robinhood was at the mercy of the industry it had vowed to upend
Answer: 1 - controversy is present.
Reasoning: The tweet highlights Robinhood’s promise to challenge Wall Street's dominance but reveals its dependence on the very system it aimed to disrupt, showing conflicts between Robinhood's mission and reality.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - controversy is not present.
Reasoning: No explicit mention of opposing opinions and disputes.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [25]:
test = pd.read_csv("test.csv")
test.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [ ]:
llm_results = []
llm_cots = []

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['controversy_llm1'] = llm_results
test['controversy_llm_cot1'] = llm_cots

In [ ]:
llm_results = []
llm_cots = []

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['controversy_llm2'] = llm_results
test['controversy_llm_cot2'] = llm_cots

In [26]:
test.groupby(['controversy_llm1','controversy_llm2'])['topic'].count()

controversy_llm1  controversy_llm2
0                 0                   365
1                 1                    34
Name: topic, dtype: int64

In [27]:
krippendorff.alpha((test['controversy_llm1'].values.tolist(), 
                    test['controversy_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

In [28]:
test.groupby(['controversy','controversy_llm1'])['topic'].count()

controversy  controversy_llm1
0            0                   365
             1                    10
1            1                    24
Name: topic, dtype: int64

In [29]:
precision = precision_score(test['controversy'], test['controversy_llm1'])
recall = recall_score(test['controversy'], test['controversy_llm1'])
f1 = f1_score(test['controversy'], test['controversy_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 0.71
Recall: 1.00
F1-score: 0.83


In [30]:
test.to_csv('test.csv')

#### Make predictions for the full dataset

In [55]:
nf = pd.read_csv("newsfactor.csv")
nf.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [56]:
MESSAGE = """Given a tweet, classify whether the controversy of an issue or event is presented in the tweet.

Note that:
*   Only explicit mentions of opposing opinions, debates, or disputes are considered. E.g., a debate about a lawsuit, disagreement with a policy.
*   Do not infer, imply, or assume controversy if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Controversy: Explicit presentation of opposing opinions, debates, or disputes. E.g., a debate about a lawsuit, disagreement with a policy.

Examples:
a) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation efforts but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 1 - controversy is present.
Reasoning: The tweet explicitly mentioned opposing opinions between the president's request and the donors' skepticism.

b) hundreds of netflix employees and supporters are expected to take part in a demonstration on wednesday to protest the streaming giant's handling of the controversy surrounding dave chappelle's recent comedy special
Answer: 1 - controversy is present.
Reasoning: The tweet explicitly mentioned opposing opinions between Netflix and its employees/ supporters with the demonstration and protest.

c) the online trading app robinhood promised to wrest the stock market away from wall street’s gatekeepers — making it as easy to put millions at risk as it is to summon an uber it turns out robinhood was at the mercy of the industry it had vowed to upend
Answer: 1 - controversy is present.
Reasoning: The tweet highlights Robinhood’s promise to challenge Wall Street's dominance but reveals its dependence on the very system it aimed to disrupt, showing conflicts between Robinhood's mission and reality.

d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
Answer: 0 - controversy is not present.
Reasoning: No explicit mention of opposing opinions and disputes.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [57]:
llm_results = []
llm_cots = []

for index, row in tqdm(nf.iterrows(), total=len(nf), desc="Classifying Tweets"):
    tweet = row['text_clean']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

nf['controversy_llm'] = llm_results
nf['controversy_llm_cot'] = llm_cots

Classifying Tweets: 100%|██████████| 9610/9610 [4:55:42<00:00,  1.85s/it]  


In [58]:
nf.to_csv('newsfactor.csv')

### Surprise

#### Prompt instruction model

In [32]:
print("Locally available Ollama models:")
print(ollama.list())

Locally available Ollama models:
models=[Model(model='llama3.1:latest', modified_at=datetime.datetime(2025, 7, 22, 16, 29, 12, 748502, tzinfo=TzInfo(+02:00)), digest='46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e', size=4920753328, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_K_M')), Model(model='deepseek-r1:latest', modified_at=datetime.datetime(2025, 7, 22, 14, 42, 30, 846205, tzinfo=TzInfo(+02:00)), digest='6995872bfe4c521a67b32da386cd21d5c6e819b6e0d62f79f64ec83be99f5763', size=5225376047, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='8.2B', quantization_level='Q4_K_M'))]


In [ ]:
messages = [
    {'role': 'user', 'content': """You are a highly precise prompt engineer. Trying to automatize part of the manual coding.
    Your goal is to generate a prompt that instructs an LLM to classify tweets.
    Below is a prompt you wrote for the varialbe scope. Now you need to adapt this prompt for the variable surprise.
    
     Prompt:   
        Given a tweet, classify whether the scope of an issue or event is presented in the tweet.

        Note that:
        *   Only explicit mentions of relevance or impact on individuals, companies, regions, or countries are considered. The time range of an issue or event, e.g., "the future of", "the month of", "a week", are not scope.
        *   Do not infer, imply, or assume scope if not explicitly stated.
        *   Do not introduce information that is not present in the text.

        Definition:
        Scope: Explicit references to the number of people, organizations, or regions involved or affected. This includes literal numbers mentioned with a verifiable source (e.g., “thousands of people”) as well as figurative speeches of a group of people or company (e.g., “all Netflix-viewers”, “Every woman living in Germany”, etc.).

        Examples:

        a) were calling for food justice in black communities and a hungerfree world for all we worked with wfpusa to suppo this united nations wfp amp samexhibition zero hunger mural by victor ash learn about our other food justice work at bhem
        Answer: 1 - scope is present.
        Reasoning: The tweet explicitly mentioned "black communities" as a figurative scope of the food justice issue.

        b) why the netflix hit squid game is resonating around the world
        Answer: 1 - scope is present.
        Reasoning: The tweet explicitly mentioned "resonating around the world" as a figurative scope of squid game.

        c) a recent survey by the brennan center for justice found 1 in 3 election officials nationwide feels unsafe at work nearly 1 in 5 called threats to their lives a jobrelated concern
        Answer: 1 - scope is present.
        Reasoning: The tweet explicitly mentioned "1 in 3 election officials nationwide" and "1 in 5", showing scope with literal numbers.

        d) learn about two different withdrawal strategies that can be used to efficiently manage your taxes in retirement
        Answer: 0 - scope is not present.
        Reasoning: No explicit mention of scope. Just mentioning "you" without explicitly show how the person is involved or affected by anything

        e) watch the livestream of amd’s ceo dr lisa su speak at computex2021 learn what is in store for amd this year and the future of pc ecosystems
        Answer: 0 - scope is not present.
        Reasoning: Although a person "ceo dr lisa su" is mentioned, the tweet did not explicitly show she is involved or affected by anything.

        THE ONLY OUTPUT should be a JSON object with two keys:
        1. `classification`: an integer (0 or 1) representing the classification result.
        2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

        Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

        Input tweet:
        '{tweet_text}'
    
    Here's the definiation of the variable surprise: surprise refers an issue or situation that is unexpected or counters expectations.
    Note. Code if the text explicitly mentions terms like “sudden”, “breaking news”, “a secret”, etc
    Here're the examples for few shot learning:
        a) a woman turning a boeing 777 into a home has revealed the secret compament where flight attendants sleep right above passengers heads
        Answer: 1 - surprise is present.
        Reasoning: The text explicitly mentions the “secret compartment,” which is a hidden or unexpected feature. So surprise is present.
        b) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
        Answer: 1 - surprise is present.
        Reasoning: The text explicitly uses the word “sudden” to describe the decision as unexpected. So surprise is present.
        c) lighting systems have a secret super power the ability to turn sites into healthybuildings
        Answer: 1 - surprise is present.
        Reasoning: The text explicitly uses the word “secret” to describe the superpower of lighting systems, revealing its surprising feature. So surprise is present.
        d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
        Answer: 0 - surprise is not present.
        Reasoning: No explicit mention of something unexpected or counter to expectations.
        e) former fed vice chairman donald kohn voiced concern on tuesday that the us central bank is not wellpositioned to deal with a rising threat of faster inflation
        Answer: 0 - surprise is not present.
        Reasoning: No explicit mention of something unexpected or counter to expectations.
        f) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
        Answer: 0 - surprise is not present.
        Reasoning: No explicit mention of something unexpected or counter to expectations. The situation is presented as factual rather than surprising.
    
    Always provide your chain of thought for generating the prompts but keep it short.
"""},
]

response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response:")
print(response['message']['content'])


Model Response:
Here's the adapted prompt for the variable `surprise`:

```
Given a tweet, classify whether an issue or situation is presented in the tweet as unexpected or counters expectations.

Note that:
*   Only explicit mentions of terms like "sudden", "breaking news", "a secret", etc. are considered.
*   Do not infer, imply, or assume surprise if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Surprise: Explicit references to an issue or situation being unexpected or counter to expectations. This includes literal mentions of terms like "sudden", "breaking news", etc., as well as figurative expressions revealing a surprising aspect.

Examples:

a) a woman turning a boeing 777 into a home has revealed the secret compament where flight attendants sleep right above passengers heads
Answer: 1 - surprise is present.
Reasoning: The text explicitly mentions the "secret compartment," which is a hidden or unexpected feature. So surpri

#### Try the prompt

In [ ]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether an issue or situation is presented in the tweet as surprising, unexpected, or counters expectations.

Note that:
*   Only explicit mentions of terms showing unexpectedness like "sudden", "breaking news", "a secret", etc. are considered.
*   Do not infer, imply, or assume surprise if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Surprise: Explicit references to an issue or situation being unexpected or counter to expectations.

Examples:
a) a woman turning a boeing 777 into a home has revealed the secret compament where flight attendants sleep right above passengers heads
Answer: 1 - surprise is present.
Reasoning: The text explicitly mentions the "secret compartment," which is a hidden or unexpected feature. So surprise is present.

b) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - surprise is present.
Reasoning: The text explicitly uses the word "sudden" to describe the decision as unexpected. So surprise is present.

c) lighting systems have a secret super power the ability to turn sites into healthybuildings
Answer: 1 - surprise is present.
Reasoning: The text explicitly uses the word "secret" to describe the superpower of lighting systems, revealing its surprising feature. So surprise is present.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - surprise is not present.
Reasoning: No explicit mention of something unexpected or counter to expectations.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

DO NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
we’re in the customer economy a great cx is essential for growth when you have consistent values both internally and externally you can quickly adjust to satisfy your evermore sophisticated clients learn how
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 0,
  "cot": "No explicit mention of something unexpected or counter to expectations."
}


In [ ]:
messages = [
    {'role': 'user', 'content': """Given a tweet, classify whether an issue or situation is presented in the tweet as surprising, unexpected, or counters expectations.

Note that:
*   Only explicit mentions of terms showing unexpectedness like "sudden", "breaking news", "a secret", etc. are considered.
*   Do not infer, imply, or assume surprise if not explicitly stated.
*   Do not introduce information that is not present in the text.

Definition:
Surprise: Explicit references to an issue or situation being unexpected or counter to expectations.

Examples:
a) a woman turning a boeing 777 into a home has revealed the secret compament where flight attendants sleep right above passengers heads
Answer: 1 - surprise is present.
Reasoning: The text explicitly mentions the "secret compartment," which is a hidden or unexpected feature. So surprise is present.

b) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - surprise is present.
Reasoning: The text explicitly uses the word "sudden" to describe the decision as unexpected. So surprise is present.

c) lighting systems have a secret super power the ability to turn sites into healthybuildings
Answer: 1 - surprise is present.
Reasoning: The text explicitly uses the word "secret" to describe the superpower of lighting systems, revealing its surprising feature. So surprise is present.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - surprise is not present.
Reasoning: No explicit mention of something unexpected or counter to expectations.

THE ONLY OUTPUT should be a JSON object with two keys:
1. `classification`: an integer (0 or 1) representing the classification result.
2. `cot`: a string containing the brief chain of thought/reasoning for the classification.

DO NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
no one expected olympian chauntelowe to get breastcancer but she followed her intuition and got medical help be proactive and ask your doctor about hereditary cancer screenings breastcancerawareness breastcancersurvivor
"""},
]
response = ollama.chat(model='llama3.1:latest', messages=messages)
print("\nModel Response (Chat):")
print(response['message']['content'])


Model Response (Chat):
{
  "classification": 1,
  "cot": "The text explicitly mentions 'no one expected' which is an explicit reference to unexpectedness."
}


#### Validate the performance

In [35]:
from ollama_classification import classify_tweet_with_ollama

In [72]:
MESSAGE = """Given a tweet, classify whether an issue or situation is presented in the tweet as surprising, unexpected, or counters expectations.

Note that:
*   Only explicit use of terms showing unexpectedness like "sudden" or "a secret", etc. are considered.
*   Just stating something or a situation is changed without framing the change clearly as unexpected in the text should not be classified as surprise.
*   Do not infer, imply, or assume something can be seen as surprising.
*   Do not introduce information that is not present in the text.

Definition:
Surprise: Explicit references to an issue or situation being unexpected or counter to expectations.

Examples:
a) a woman turning a boeing 777 into a home has revealed the secret compament where flight attendants sleep right above passengers heads
Answer: 1 - surprise is present.
Reasoning: The text explicitly mentions the "secret compartment," which is a hidden or unexpected feature. So surprise is present.

b) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - surprise is present.
Reasoning: The text explicitly uses the word "sudden" to describe the decision as unexpected. So surprise is present.

c) lighting systems have a secret super power the ability to turn sites into healthybuildings
Answer: 1 - surprise is present.
Reasoning: The text explicitly uses the word "secret" to describe the superpower of lighting systems, revealing its surprising feature. So surprise is present.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - surprise is not present.
Reasoning: No explicit mention of something unexpected or counter to expectations. Although the european central bank changed the inflation target, it is not presented as surprising or unexpected.

e) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 0 - surprise is not present.
Reasoning: No explicit mention of something unexpected or counter to expectations. The situation is presented as factual rather than surprising.


THE ONLY OUTPUT should be a JSON object with two keys:
1. `cot`: a string containing the brief chain of thought/reasoning for the classification.
2. `classification`: an integer (0 or 1) representing the classification result.


Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [60]:
val = pd.read_csv("val.csv")
val.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [73]:
llm_results = []
llm_cots = []

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['surprise_llm1'] = llm_results
val['surprise_llm_cot1'] = llm_cots

Classifying Tweets: 100%|██████████| 81/81 [02:56<00:00,  2.18s/it]


In [68]:
llm_results = []
llm_cots = []

for index, row in tqdm(val.iterrows(), total=len(val), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

val['surprise_llm2'] = llm_results
val['surprise_llm_cot2'] = llm_cots

Classifying Tweets: 100%|██████████| 81/81 [02:47<00:00,  2.07s/it]


In [69]:
val.groupby(['surprise_llm1','surprise_llm2'])['topic'].count()

surprise_llm1  surprise_llm2
0              0                78
1              1                 3
Name: topic, dtype: int64

In [70]:
krippendorff.alpha((val['surprise_llm1'].values.tolist(), 
                    val['surprise_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

In [74]:
val.groupby(['surprise','surprise_llm1'])['topic'].count()

surprise  surprise_llm1
0         0                78
          1                 1
1         1                 2
Name: topic, dtype: int64

In [75]:
precision = precision_score(val['surprise'], val['surprise_llm1'])
recall = recall_score(val['surprise'], val['surprise_llm1'])
f1 = f1_score(val['surprise'], val['surprise_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 0.67
Recall: 1.00
F1-score: 0.80


In [71]:
val.to_csv('val.csv')

#### Test the final prompt

In [ ]:
MESSAGE = """Given a tweet, classify whether an issue or situation is presented in the tweet as surprising, unexpected, or counters expectations.

Note that:
*   Only explicit use of terms showing unexpectedness like "sudden" or "a secret", etc. are considered.
*   Just stating something or a situation is changed without framing the change clearly as unexpected in the text should not be classified as surprise.
*   Do not infer, imply, or assume something can be seen as surprising.
*   Do not introduce information that is not present in the text.

Definition:
Surprise: Explicit references to an issue or situation being unexpected or counter to expectations.

Examples:
a) a woman turning a boeing 777 into a home has revealed the secret compament where flight attendants sleep right above passengers heads
Answer: 1 - surprise is present.
Reasoning: The text explicitly mentions the "secret compartment," which is a hidden or unexpected feature. So surprise is present.

b) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - surprise is present.
Reasoning: The text explicitly uses the word "sudden" to describe the decision as unexpected. So surprise is present.

c) lighting systems have a secret super power the ability to turn sites into healthybuildings
Answer: 1 - surprise is present.
Reasoning: The text explicitly uses the word "secret" to describe the superpower of lighting systems, revealing its surprising feature. So surprise is present.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - surprise is not present.
Reasoning: No explicit mention of something unexpected or counter to expectations. Although the european central bank changed the inflation target, it is not presented as surprising or unexpected.

e) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 0 - surprise is not present.
Reasoning: No explicit mention of something unexpected or counter to expectations. The situation is presented as factual rather than surprising.


THE ONLY OUTPUT should be a JSON object with two keys:
1. `cot`: a string containing the brief chain of thought/reasoning for the classification.
2. `classification`: an integer (0 or 1) representing the classification result.


Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [76]:
test = pd.read_csv("test.csv")
test.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [ ]:
llm_results = []
llm_cots = []

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['surprise_llm1'] = llm_results
test['surprise_llm_cot1'] = llm_cots

In [ ]:
llm_results = []
llm_cots = []

for index, row in tqdm(test.iterrows(), total=len(test), desc="Classifying Tweets"):
    tweet = row['rep_doc']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

test['surprise_llm2'] = llm_results
test['surprise_llm_cot2'] = llm_cots

In [77]:
test.groupby(['surprise_llm1','surprise_llm2'])['topic'].count()

surprise_llm1  surprise_llm2
0              0                395
1              1                  4
Name: topic, dtype: int64

In [78]:
krippendorff.alpha((test['surprise_llm1'].values.tolist(), 
                    test['surprise_llm2'].values.tolist()), level_of_measurement="nominal")

np.float64(1.0)

In [79]:
test.groupby(['surprise','surprise_llm1'])['topic'].count()

surprise  surprise_llm1
0         0                395
          1                  2
1         1                  2
Name: topic, dtype: int64

In [80]:
precision = precision_score(test['surprise'], test['surprise_llm1'])
recall = recall_score(test['surprise'], test['surprise_llm1'])
f1 = f1_score(test['surprise'], test['surprise_llm1'])

# Print individual scores
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Precision: 0.50
Recall: 1.00
F1-score: 0.67


In [ ]:
test.to_csv('test.csv')

#### Make predictions for the full dataset

In [81]:
nf = pd.read_csv("newsfactor.csv")
nf.drop(labels = ["Unnamed: 0"], axis = 1, inplace = True)

In [82]:
MESSAGE = """Given a tweet, classify whether an issue or situation is presented in the tweet as surprising, unexpected, or counters expectations.

Note that:
*   Only explicit use of terms showing unexpectedness like "sudden" or "a secret", etc. are considered.
*   Just stating something or a situation is changed without framing the change clearly as unexpected in the text should not be classified as surprise.
*   Do not infer, imply, or assume something can be seen as surprising.
*   Do not introduce information that is not present in the text.

Definition:
Surprise: Explicit references to an issue or situation being unexpected or counter to expectations.

Examples:
a) a woman turning a boeing 777 into a home has revealed the secret compament where flight attendants sleep right above passengers heads
Answer: 1 - surprise is present.
Reasoning: The text explicitly mentions the "secret compartment," which is a hidden or unexpected feature. So surprise is present.

b) texas gov abbott’s sudden decision this week to lift the mask mandate and other coronavirus restrictions has undermined the ability of private companies to enforce their own rules for protecting staffers and customers security expes say
Answer: 1 - surprise is present.
Reasoning: The text explicitly uses the word "sudden" to describe the decision as unexpected. So surprise is present.

c) lighting systems have a secret super power the ability to turn sites into healthybuildings
Answer: 1 - surprise is present.
Reasoning: The text explicitly uses the word "secret" to describe the superpower of lighting systems, revealing its surprising feature. So surprise is present.

d) the european central bank took a step in the federal reserve’s direction with its new inflation target of 2
Answer: 0 - surprise is not present.
Reasoning: No explicit mention of something unexpected or counter to expectations. Although the european central bank changed the inflation target, it is not presented as surprising or unexpected.

e) president jair bolsonaro of brazil wants the international community to pledge billions of dollars to pay for conservation effos but donors are skeptical under his watch deforestation in the amazon has risen to the highest level in over a decade
Answer: 0 - surprise is not present.
Reasoning: No explicit mention of something unexpected or counter to expectations. The situation is presented as factual rather than surprising.


THE ONLY OUTPUT should be a JSON object with two keys:
1. `cot`: a string containing the brief chain of thought/reasoning for the classification.
2. `classification`: an integer (0 or 1) representing the classification result.


Do NOT include any other text, explanations, or Markdown code block fences (like ```json or ```) in your response. Output ONLY the JSON object.

Input tweet:
'{tweet_text}'
"""

In [83]:
llm_results = []
llm_cots = []

for index, row in tqdm(nf.iterrows(), total=len(nf), desc="Classifying Tweets"):
    tweet = row['text_clean']
    llm_classification, llm_cot = classify_tweet_with_ollama(text=tweet, model="llama3.1", message=MESSAGE)
    llm_results.append(llm_classification)
    llm_cots.append(llm_cot)

nf['surprise_llm'] = llm_results
nf['surprise_llm_cot'] = llm_cots

Classifying Tweets: 100%|██████████| 9610/9610 [5:21:00<00:00,  2.00s/it]  


In [84]:
nf.to_csv('newsfactor.csv')